
---

# Motivation

### What is your dataset?

We selected a comprehensive dataset of European Parliament voting records. This dataset captures how each Member of the European Parliament (MEP) voted on proposed legislation, along with detailed information about the legislation itself, the MEPs, and their party affiliations. The dataset includes two main file types:

* **RCV (Roll Call Votes):** Contains individual vote records for each MEP, including their country, national party, and European Parliamentary Group (EPG).
* **Voted Docs:** Provides metadata on the legislation being voted on, including the vote date, outcome, and associated policy area.

The data is divided across four parliamentary sessions:

* EP6 (2004–2009)
* EP7 (2009–2014)
* EP8 (2014–2019)
* EP9 (2019–2022)

To enable longitudinal analysis, we merged data from all four sessions. This required extensive preprocessing to reconcile differences in schema and formatting across sessions.

### Why did you choose this particular dataset?

We chose this dataset because it offers a rich foundation for analyzing trends in European politics over time. It enables us to explore whether broader political shifts—such as increasing polarization or the rise of right-leaning ideologies—are observable in parliamentary voting behavior.

### What was your goal for the end user's experience?

Our goal is to provide users with an accessible interface to explore aggregated voting patterns and to present clear, data-driven insights into evolving political dynamics within the European Parliament. We aim to make it easy for users to identify trends related to polarization, such as which parties or policy areas are becoming more divisive, and which remain broadly supported. Ultimately, we want to support informed reflection on how shifts in ideology are shaping legislative decision-making at the EU level.

---


---

# Basic Stats

### Preprocessing

To analyze voting trends in the European Parliament across four sessions (EP6–EP9), we consolidated roll-call vote data (`RCV`) with metadata on each legislative item (`Voted Docs`). Due to inconsistencies across sessions in schema, date formats, and naming conventions, several preprocessing steps were necessary:

* **Name Standardization:** MEP names were cleaned to ensure consistency across sessions, accounting for formatting, punctuation, and Unicode differences.
* **Text Normalization:** Policy area fields and party/EPG names were cleaned to remove noise (e.g., punctuation, inconsistent capitalization).
* **Date Parsing:** Dates appeared in both `dd.mm.yyyy` and `yyyy-mm-dd` formats. A custom parser ensured accurate conversion to a uniform `datetime` format.
* **Schema Harmonization:** Since column names and vote identifiers varied by session (e.g., `euro_act_id` in EP6 vs `Vote ID` in later sessions), we created session-aware mappings to unify data.
* **Party Group Mapping:** EP political group (`EPG`) names were mapped to their common abbreviations (e.g., “Group of the European People’s Party...” → `EPP`) to allow for consistent comparison.
* **Missing Values:** Non-informative entries (e.g., empty vote fields or unknown policy areas) were filtered or imputed based on context.

These steps allowed us to generate a unified dataset spanning 2004–2022 with over **4.8 million individual votes**.

### Dataset Statistics

Key observations from the cleaned data:

* 📅 **Time Coverage:** The dataset spans four European Parliament terms:

  * EP6: 2004–2009
  * EP7: 2009–2014
  * EP8: 2014–2019
  * EP9: 2019–2022

* 🧑‍🤝‍🧑 **MEPs Involved:** \~1,300 unique MEPs from all 27 EU member states.

* 📄 **Votes Recorded:**

  * \~26,000 roll-call votes
  * \~4.8 million MEP-level voting entries (rows)

* 🗳️ **Vote Distribution (Sample):**

  | Vote      | Meaning                    | % of Total     |
  | --------- | -------------------------- | -------------- |
  | 1         | For                        | \~48%          |
  | 2         | Against                    | \~27%          |
  | 3         | Abstain                    | \~12%          |
  | 4 / 5 / 6 | Absent / No vote / Excused | \~13% combined |

* 🏛️ **Top Policy Areas:** After cleaning and consolidating, common topics included:

  * Environment
  * Budget and financial affairs
  * Foreign relations
  * Digital and industry regulation
  * Civil rights and rule of law

* 🧭 **EP Group Participation:** Most votes came from major groups like `EPP`, `S&D`, `Renew`, `ID`, `Greens/EFA`, and `The Left`.

---


In [1]:
import pandas as pd

def clean_name(first_name, last_name):
    import unicodedata
    
    if not isinstance(first_name, str):
        first_name = str(first_name) if first_name is not None else ""
    if not isinstance(last_name, str):
        last_name = str(last_name) if last_name is not None else ""
    
    first_name = first_name.lower().strip()
    last_name = last_name.lower().strip()
    
    def normalize_chars(text):
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
        return text
    
    first_name = normalize_chars(first_name)
    last_name = normalize_chars(last_name)
    
    for char in ['-', "'", "`", ".", ",", "&", "'"]:  # Added apostrophe variants
        first_name = first_name.replace(char, ' ')
        last_name = last_name.replace(char, ' ')
    
    while '  ' in first_name:
        first_name = first_name.replace('  ', ' ')
    while '  ' in last_name:
        last_name = last_name.replace('  ', ' ')
        
    first_name = ' '.join(word.capitalize() for word in first_name.split())
    last_name = ' '.join(word.capitalize() for word in last_name.split())
    
    full_name = f"{first_name} {last_name}".strip()
    
    return full_name

def clean_text(text):
    if not isinstance(text, str):
        return text
  
    text = text.lower()
    
    for char in ['&', ',', '-']:
        text = text.replace(char, ' ')
    
    text = text.replace(' and ', ' ')
    
    while '  ' in text:
        text = text.replace('  ', ' ')
    
    return text.strip()    

def process_ep_voting_data(rcv_files, voted_docs_files):

    if len(rcv_files) != len(voted_docs_files):
        raise ValueError("The lists of RCV files and Voted docs files must have the same length")
     
    all_data = []
    
    for i, (rcv_file, voted_doc_file) in enumerate(zip(rcv_files, voted_docs_files)):
        print(f"Processing files {i+1}/{len(rcv_files)}: {rcv_file} and {voted_doc_file}")
        
        if "EP6" in rcv_file:
            ep_session = "EP6"
            vote_start_index = 10
            rcv_data = pd.read_excel(rcv_file, header=1)
        elif "EP7" in rcv_file:
            ep_session = "EP7"
            vote_start_index = 9
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
        elif "EP8" in rcv_file:
            ep_session = "EP8"
            vote_start_index = 9
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
        elif "EP9" in rcv_file:
            ep_session = "EP9"
            vote_start_index = 10
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
        else:
            ep_session = "Unknown"
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
            print("UNKNOWN SESSION")

        rcv_data = rcv_data.dropna(how='all')
        
        voted_docs = pd.read_excel(voted_doc_file)


        # Get vote columns headers (index)
        vote_columns = rcv_data.columns[vote_start_index:].tolist()
       
        votes_df = process_votes_ep(rcv_data, voted_docs, vote_columns, ep_session=ep_session)

        print(f"Should be around: {len(rcv_data) * len(voted_docs)}")
        print(f"Got: {len(votes_df)}")      

        # Add EP session information
        votes_df['ep_session'] = ep_session
        
        # Append to the list of results
        all_data.append(votes_df)
    
    # Concatenate all dataframes
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Perform final cleaning
    return combined_df


def process_votes_ep(rcv_data, voted_docs, vote_columns, ep_session = None):
    """Process voting data for EP7, EP8, EP9 sessions"""

    total_skipped = 0

    if ep_session == 'EP6':
        date = 'date'
        title = 'title'
        policy_area = 'main_policy_name'
        vote_id_key = 'euro_act_id'
        author = 'author_name'

        mep_id_key = 'WebisteEpID'

    else:
        date = 'Date'
        title = 'Title'
        policy_area = 'De'
        vote_id_key = 'Vote ID'
        author = 'Author'

        mep_id_key = 'WebisteEpID'

        if ep_session == 'EP7':
            mep_id_key = 'MEP ID'

        if ep_session == 'EP8':
            policy_area = "De/Policy area"

        elif ep_session == 'EP9':
            policy_area = 'Policy area'
  
    
    # Create a dictionary to map vote IDs to vote information
    vote_info = {}
    for _, row in voted_docs.iterrows():

        vote_info[str(row[vote_id_key])] = {
            'date': row[date],
            'title': row[title],
            'policy_area': row[policy_area],
            'author': author,
        }
    
    # Create a list to store results
    results = []
    
    # Process each MEP's votes
    for _, mep_row in rcv_data.iterrows():
        country = mep_row['Country']
        party = mep_row['Party']
        epg = mep_row['EPG']

        first_name = mep_row['Fname']
        last_name = mep_row['Lname']
        
        mep_id = mep_row[mep_id_key]
    
        # Process each vote for this MEP
        for vote_col in vote_columns:
            
            vote_col = str(vote_col)
            vote_code = f'{ep_session}-{vote_col}' 
            
            if vote_col not in vote_info:
                total_skipped += 1
                continue
            
            try:
                mep_vote = mep_row[str(vote_col)]
            except Exception as e:
                mep_vote = mep_row[int(vote_col)]
            
            if mep_vote == 0:
                continue
                
            info = vote_info[vote_col]
            
            results.append({
                'full name': clean_name(first_name, last_name),
                'country': country,
                'national_party': party,
                'epg': epg,
                'mep_id': mep_id,
                'vote_code': vote_code,
                'vote': mep_vote,
                'date': info['date'],
                'title': info['title'],
                'policy_area': clean_text(info['policy_area']),
            })
    
    print(f"Were not able to match: {total_skipped} votes")
    return pd.DataFrame(results)


In [20]:
import pandas as pd
import numpy as np

def parse_mixed_dates(date_str):
    
    if not isinstance(date_str, str):
        date_str = str(date_str)
    
    date_str = date_str.strip()
    
    try:
        # Check for dates with time component
        if " 00:00:00" in date_str:
            date_str = date_str.replace(" 00:00:00", "")

        if date_str == '18 ian 2007':
            return pd.to_datetime('2007-01-18')
        
        if '.' in date_str:
            # Parse as dd.mm.yyyy
            return pd.to_datetime(date_str, format='%d.%m.%Y')
        elif '-' in date_str:
            # Parse as yyyy-mm-dd
            return pd.to_datetime(date_str, format='%Y-%m-%d')
        elif '/' in date_str:
            try:
                # First try %d/%m/%Y (day/month/4-digit year)
                return pd.to_datetime(date_str, format='%d/%m/%Y')
            except ValueError:
                try:
                    # Then try %d/%m/%y (day/month/2-digit year)
                    return pd.to_datetime(date_str, format='%d/%m/%y')
                except ValueError:
                    # Fall back to pandas default parser with dayfirst=True
                    return pd.to_datetime(date_str, dayfirst=True)
        else:
            # For formats we haven't explicitly handled, use pandas' flexible parser
            return pd.to_datetime(date_str)
            
    except Exception as e:
        print(f"Error parsing date '{date_str}': {e}")
        return pd.NaT  # In case of error
    

def clean_combined_data(df):
    
    df['date'] = df['date'].astype(str).apply(parse_mixed_dates)
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    
    df['policy_area_cleaned'] = df['policy_area'].str.strip().str.lower()

    # Map EPG to ture names:
    epg_mapping = {
        "Group of the European People's Party (Christian Democrats)": 'EPP',
        "Group of the European People's Party (Christian Democrats) and European Democrats": 'EPP',
        'Socialist Group in the European Parliament': 'S&D',
        'Group of the Progressive Alliance of Socialists and Democrats in the European Parliament': 'S&D',
        'Confederal Group of the European United Left - Nordic Green Left': 'The Left',
        'Group of the Greens/European Free Alliance': 'Greens/EFA',
        'Independence/Democracy Group': 'IDG',
        'Europe of freedom and democracy Group': 'IDG',
        'Europe of Freedom and Direct Democracy Group': 'IDG',
        'Europe of Nations and Freedom Group': 'ID',
        'European Conservatives and Reformists Group': 'ECR',
        'Non-attached Members': 'NI',
        'Group of the Alliance of Liberals and Democrats for Europe' : 'REG'
    }
    df['epg'] = df['epg'].replace(epg_mapping)

    # Merge PA 
    df['policy_area'] = df['policy_area'].replace('regioanal development', 'regional development')
    df['policy_area'] = df['policy_area'].replace('economic monetary affairs', 'economics')
    
    return df



In [3]:

voted_docs_files = ["VoteWatch-EP-voting-data_2004-2022/EP6_Voted docs.xlsx", "VoteWatch-EP-voting-data_2004-2022/EP7_Voted docs.xlsx", "VoteWatch-EP-voting-data_2004-2022/EP8_Voted docs.xlsx", "VoteWatch-EP-voting-data_2004-2022/EP9_Voted docs.xlsx"]
rcv_files = ["VoteWatch-EP-voting-data_2004-2022/EP6_RCVs_2022_06_13.xlsx", "VoteWatch-EP-voting-data_2004-2022/EP7_RCVs_2014_06_19.xlsx", "VoteWatch-EP-voting-data_2004-2022/EP8_RCVs_2019_06_25.xlsx", "VoteWatch-EP-voting-data_2004-2022/EP9_RCVs_2022_06_22.xlsx"]

df = process_ep_voting_data(rcv_files, voted_docs_files)

print('Saving uncleaned data')

# Save the combined dataframe
output_file = "ep_voting_data_combined_raw.csv"
df.to_csv(output_file, index=False)

print('Cleaning data')
df = clean_combined_data(df)

print('Saving cleaned data')

output_file = "ep_voting_data_combined_clean.csv"
df.to_csv(output_file, index=False)


Processing files 1/4: VoteWatch-EP-voting-data_2004-2022/EP6_RCVs_2022_06_13.xlsx and VoteWatch-EP-voting-data_2004-2022/EP6_Voted docs.xlsx
Were not able to match: 0 votes
Should be around: 5827060
Got: 4759840
Processing files 2/4: VoteWatch-EP-voting-data_2004-2022/EP7_RCVs_2014_06_19.xlsx and VoteWatch-EP-voting-data_2004-2022/EP7_Voted docs.xlsx
Were not able to match: 0 votes
Should be around: 5937733
Got: 5233859
Processing files 3/4: VoteWatch-EP-voting-data_2004-2022/EP8_RCVs_2019_06_25.xlsx and VoteWatch-EP-voting-data_2004-2022/EP8_Voted docs.xlsx
Were not able to match: 0 votes
Should be around: 8796216
Got: 7696506
Processing files 4/4: VoteWatch-EP-voting-data_2004-2022/EP9_RCVs_2022_06_22.xlsx and VoteWatch-EP-voting-data_2004-2022/EP9_Voted docs.xlsx


/Users/markusswegmark/dev/-socialdata2025-MMC-/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Were not able to match: 0 votes
Should be around: 10915249
Got: 9520348
Saving uncleaned data
Cleaning data


In [4]:
# Get all column headers as a list
headers_list = df.columns.tolist()
print("Column headers:")
print(headers_list)

Column headers:
['full name', 'country', 'national_party', 'epg', 'mep_id', 'vote_code', 'vote', 'date', 'title', 'policy_area', 'ep_session', 'year', 'month', 'policy_area_cleaned']


In [1]:
import pandas as pd

try:
    df
except NameError:
    df = pd.read_csv('ep_voting_data_combined_clean.csv', dtype=str)


In [21]:
df_cleaned = clean_combined_data(df)

/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_52017/818729727.py:35: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date_str, dayfirst=True)


In [2]:
headers_list = list(df.columns)
print(headers_list)

['full name', 'country', 'national_party', 'epg', 'mep_id', 'vote_code', 'vote', 'date', 'title', 'policy_area', 'ep_session', 'year', 'month', 'policy_area_cleaned']


In [3]:
# Step 1: Find EPGs present in all years
years = df['year'].unique()
epgs_by_year = [set(df[df['year'] == year]['epg'].dropna().unique()) for year in years]

print(epgs_by_year)
# Step 2: Find common EPGs
common_epgs = set.intersection(*epgs_by_year)

# Step 3: Print the result
print(f"EPGs present in all {len(years)} years:")
print(sorted(common_epgs))

[{'NI', 'Union for Europe of the Nations Group', 'The Left', 'S&D', 'IDG', 'Greens/EFA', 'REG', 'EPP'}, {'NI', 'Union for Europe of the Nations Group', 'The Left', 'S&D', 'IDG', 'Greens/EFA', 'REG', 'EPP'}, {'NI', 'Union for Europe of the Nations Group', 'The Left', 'S&D', 'IDG', 'Greens/EFA', 'REG', 'EPP'}, {'NI', 'Union for Europe of the Nations Group', 'The Left', 'S&D', 'IDG', 'Greens/EFA', 'REG', 'EPP'}, {'NI', 'Union for Europe of the Nations Group', 'The Left', 'S&D', 'IDG', 'Greens/EFA', 'REG', 'EPP'}, {'NI', 'Union for Europe of the Nations Group', 'The Left', 'S&D', 'IDG', 'Greens/EFA', 'REG', 'ECR', 'EPP'}, {'NI', 'S&D', 'The Left', 'IDG', 'Greens/EFA', 'REG', 'ECR', 'EPP'}, {'NI', 'S&D', 'The Left', 'IDG', 'Greens/EFA', 'REG', 'ECR', 'EPP'}, {'NI', 'S&D', 'The Left', 'IDG', 'Greens/EFA', 'REG', 'ECR', 'EPP'}, {'NI', 'S&D', 'The Left', 'IDG', 'Greens/EFA', 'REG', 'ECR', 'EPP'}, {'NI', 'S&D', 'The Left', 'IDG', 'Greens/EFA', 'REG', 'ID', 'ECR', 'EPP'}, {'NI', 'The Left', 'S&D

## Create simmilarity matrix between EPGs for each year and policy area

In [15]:
from itertools import combinations

def agreement_index(votes1, votes2):
    """
    Calculate agreement between two voting groups based on vote percentage distributions.
    
    Parameters:
    - votes1, votes2: Series of vote counts indexed by vote type (1=for, 2=against, 3=abstention, etc.)
    
    Returns:
    - Similarity score between 0 and 1, based on overlap of percentage distributions
    """
    # Calculate total votes for each group
    total_votes_1 = sum(votes1)
    total_votes_2 = sum(votes2)
    
    total_agreement = 0
    
    # For each vote type, calculate the proportion of agreement based on percentages
    for i in range(len(votes1)):
        # Calculate percentages
        pct1 = votes1[i] / total_votes_1
        pct2 = votes2[i] / total_votes_2
        
        # Add minimum percentage as the agreement for this vote type
        total_agreement += min(pct1, pct2)
    
    # The sum of all minimum percentages directly represents the overlap
    # between the two distributions (ranges from 0 to 1)
    return total_agreement

# Modified code to use the new similarity function
df['epg'] = df['epg'].astype(str)
df['year'] = df['year'].astype(float).astype(int)
df_year = df.groupby(['year', 'policy_area'])
epgs = sorted(list(common_epgs))
similarity_matrices = {}

# Track missing data
missing_data_count = 0
total_combinations = 0

for name, group in df_year:
    year, policy_area = name 
    year = int(year)
    sim_matrix = pd.DataFrame(index=epgs, columns=epgs)
    
    # Track missing data for this year/policy_area
    missing_in_current = 0
    total_in_current = 0
    
    # Calculate similarities between all EPG pairs
    for epg1, epg2 in combinations(epgs, 2):
        total_combinations += 1
        total_in_current += 1
        
        ep1_votes_series = group[group['epg'] == epg1]['vote'].value_counts()
        ep2_votes_series = group[group['epg'] == epg2]['vote'].value_counts()

        # Initialize arrays with zeros for vote types 1, 2, and 3
        ep1_votes = [0, 0, 0]  # Index 0 for vote value 1, index 1 for vote value 2, etc.
        ep2_votes = [0, 0, 0]

        # Fill in the counts from the Series, handling both int and float vote types
        for vote_type, count in ep1_votes_series.items():
            vote_index = int(float(vote_type)) - 1  # Convert vote type (1,2,3) to index (0,1,2)
            if 0 <= vote_index <= 2:  # Only include vote types 1, 2, and 3
                ep1_votes[vote_index] = count

        for vote_type, count in ep2_votes_series.items():
            vote_index = int(float(vote_type)) - 1
            if 0 <= vote_index <= 2:
                ep2_votes[vote_index] = count

        
        # Check if any relevant votes exist for both EPGs
        has_votes_1 = sum(ep1_votes) != 0
        has_votes_2 = sum(ep2_votes) != 0
        
        if not has_votes_1 or not has_votes_2:
            missing_data_count += 1
            missing_in_current += 1
            similarity = 0  # No relevant votes, similarity is 0
        else:
            similarity = agreement_index(ep1_votes, ep2_votes)
            
        sim_matrix.loc[epg1, epg2] = similarity
        sim_matrix.loc[epg2, epg1] = similarity  # Matrix is symmetric
    
    # Set diagonal to 1 (self-similarity)
    for epg in epgs:
        sim_matrix.loc[epg, epg] = 1.0
            
    # Store the matrix
    if policy_area not in similarity_matrices:
        similarity_matrices[policy_area] = {}
    similarity_matrices[policy_area][year] = sim_matrix
    
    # Print summary for this year/policy_area
    if missing_in_current > 0:
        print(f"Year {year}, Policy Area '{policy_area}': {missing_in_current}/{total_in_current} EPG pairs have missing vote data ({missing_in_current/total_in_current:.1%})")

# Print overall summary
print(f"\nOverall: {missing_data_count}/{total_combinations} EPG pairs have missing vote data ({missing_data_count/total_combinations:.1%})")


Overall: 0/8106 EPG pairs have missing vote data (0.0%)


## Create PCA, tranform with procrustes and animate over time for paramter policy area

In [35]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.spatial import procrustes
from bokeh.plotting import figure, save, output_file
from bokeh.models import ColumnDataSource, LabelSet, Slider, CustomJS, Button, Range1d
from bokeh.layouts import column, row
from bokeh.palettes import Category10

# Set output to an HTML file
output_file("epg_clustering_animation_better_zoom.html")

# Select a single policy area to visualize
policy_area = 'budget'  # Change this to any policy area you want

# Get all available years for this policy area
years = sorted(similarity_matrices[policy_area].keys())

# Get EPGs from the first year
epgs = list(similarity_matrices[policy_area][years[0]].index)

# Create color mapping for EPGs
epg_colors = {}
palette = Category10[10]
for i, epg in enumerate(epgs):
    epg_colors[epg] = palette[i % len(palette)]

# Function to perform dimensionality reduction on a similarity matrix
def get_coordinates(similarity_matrix, method='pca'):
    # Convert similarity to distance matrix
    distance_matrix = 1 - similarity_matrix
    
    # Convert to numpy array
    X = distance_matrix.values
    
    # Apply dimensionality reduction
    if method == 'pca':
        model = PCA(n_components=2)
        result = model.fit_transform(X)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Create DataFrame with results
    df_result = pd.DataFrame({
        'x': result[:, 0],
        'y': result[:, 1],
        'epg': distance_matrix.index,
        'color': [epg_colors.get(epg, '#000000') for epg in distance_matrix.index]
    })
    
    return df_result

# Function to align coordinates with reference using Procrustes analysis
def align_coordinates(target_df, reference_df):
    # Get common EPGs
    common_epgs = set(target_df['epg']).intersection(set(reference_df['epg']))
    
    if len(common_epgs) < 2:
        # Not enough common points to align
        return target_df
    
    # Extract coordinates for common EPGs
    target_coords = np.array([target_df[target_df['epg'] == epg][['x', 'y']].values[0] for epg in common_epgs])
    reference_coords = np.array([reference_df[reference_df['epg'] == epg][['x', 'y']].values[0] for epg in common_epgs])
    
    # Perform Procrustes analysis to align target to reference
    mtx1, mtx2, disparity = procrustes(reference_coords, target_coords)
    
    # Create transformation matrix (scale, rotation, reflection)
    scale = np.sqrt(np.sum(mtx1[0]**2)) / np.sqrt(np.sum(target_coords[0]**2))
    
    # Apply transformation to all points in target_df
    result_df = target_df.copy()
    coords = result_df[['x', 'y']].values
    
    # Scale and center (simplified Procrustes)
    coords_scaled = coords * scale
    
    # Get centroids
    target_centroid = np.mean(target_coords, axis=0)
    reference_centroid = np.mean(reference_coords, axis=0)
    
    # Translate
    coords_transformed = coords_scaled - target_centroid + reference_centroid
    
    # Update dataframe
    result_df['x'] = coords_transformed[:, 0]
    result_df['y'] = coords_transformed[:, 1]
    
    return result_df

# Compute coordinates for the first year (reference)
method = 'pca'  # PCA is more stable for small numbers of points
reference_data = get_coordinates(similarity_matrices[policy_area][years[0]], method=method)

# Compute and align coordinates for all years
aligned_data = {}
for year in years:
    # Compute initial coordinates
    year_data = get_coordinates(similarity_matrices[policy_area][year], method=method)
    
    # Align with reference
    if year == years[0]:
        aligned_data[year] = year_data  # Reference year doesn't need alignment
    else:
        aligned_data[year] = align_coordinates(year_data, reference_data)

# Find the overall range of data across all years to set consistent plot boundaries
all_x = []
all_y = []
for year_data in aligned_data.values():
    all_x.extend(year_data['x'])
    all_y.extend(year_data['y'])

x_min, x_max = min(all_x), max(all_x)
y_min, y_max = min(all_y), max(all_y)

# Add padding (200% zoom - 50% padding on each side)
padding_x = (x_max - x_min) * 0.5
padding_y = (y_max - y_min) * 0.5
x_range = (x_min - padding_x, x_max + padding_x)
y_range = (y_min - padding_y, y_max + padding_y)

# Create initial plot with first year
current_year = years[0]
init_data = aligned_data[current_year]

# Create ColumnDataSource
source = ColumnDataSource(init_data)

# Create the figure with fixed range
p = figure(width=700, height=500, 
           title=f'EPG Clustering - {policy_area} ({current_year})',
           tools="pan,wheel_zoom,box_zoom,reset,save",
           x_range=Range1d(x_range[0], x_range[1]),
           y_range=Range1d(y_range[0], y_range[1]))

# Add scatter points
p.circle('x', 'y', source=source, size=15, color='color', alpha=0.8, 
         line_color='black', line_width=1)

# Add labels
labels = LabelSet(x='x', y='y', text='epg', source=source,
                 text_font_size='10pt', text_color='black',
                 x_offset=5, y_offset=5)
p.add_layout(labels)

# Prepare data for JavaScript
js_data = {}
for year in years:
    js_data[str(year)] = aligned_data[year].to_dict('list')

# Create a slider for years
year_slider = Slider(title="Year", start=0, end=len(years)-1, value=0, step=1, width=600)

# Create play/pause button
play_button = Button(label="▶️ Play", button_type="success", width=100)

# JavaScript callback for slider
slider_callback = CustomJS(args=dict(source=source, p=p, years=years, 
                                   policy_area=policy_area, data=js_data), code="""
    // Get the selected year index
    const yearIndex = cb_obj.value;
    const year = years[yearIndex];
    
    // Update data from precomputed results
    const year_data = data[year];
    
    // Update the source data
    source.data['x'] = year_data['x'];
    source.data['y'] = year_data['y'];
    source.data['epg'] = year_data['epg'];
    source.data['color'] = year_data['color'];
    
    // Update title
    p.title.text = `EPG Clustering - ${policy_area} (${year})`;
    
    // Trigger update
    source.change.emit();
""")

# Connect callback to slider
year_slider.js_on_change('value', slider_callback)

# Animation callback for play button
animation_callback = CustomJS(args=dict(slider=year_slider, button=play_button), code="""
    if (button.label === "▶️ Play") {
        // Start animation
        button.label = "⏸️ Pause";
        
        // Function to increment slider
        function animate_slider() {
            if (button.label === "⏸️ Pause") {
                let current = slider.value;
                let next = current + 1;
                
                // Loop back to beginning if at the end
                if (next > slider.end) {
                    next = slider.start;
                }
                
                // Update slider value (this will trigger the slider callback)
                slider.value = next;
                
                // Schedule next update
                window.setTimeout(animate_slider, 1000);  // 1 second interval
            }
        }
        
        // Start animation
        animate_slider();
    } else {
        // Pause animation
        button.label = "▶️ Play";
    }
""")

# Connect animation callback to play button
play_button.js_on_click(animation_callback)

# Create layout
layout = column(
    row(year_slider, play_button),
    p
)

# Save the visualization to an HTML file
save(layout)

print("Visualization saved to 'epg_clustering_animation.html'. Open this file in a web browser to view the animation.")

Visualization saved to 'epg_clustering_animation_better_zoom.html'. Open this file in a web browser to view the animation.


## Aggregated over all policy areas

In [37]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.spatial import procrustes
from bokeh.plotting import figure, save, output_file
from bokeh.models import ColumnDataSource, LabelSet, Slider, CustomJS, Button, Range1d
from bokeh.layouts import column, row
from bokeh.palettes import Category10

# Set output to an HTML file
output_file("epg_clustering_all_policies_merged.html")

# Merge data from all policy areas for each year
merged_similarity_matrices = {}

# Get all unique years across all policy areas
all_years = set()
for policy_area in similarity_matrices:
    all_years.update(similarity_matrices[policy_area].keys())
all_years = sorted(all_years)

# Get all unique EPGs
all_epgs = set()
for policy_area in similarity_matrices:
    for year in similarity_matrices[policy_area]:
        all_epgs.update(similarity_matrices[policy_area][year].index)
all_epgs = sorted(list(all_epgs))

# Merge data for each year by averaging similarity scores across policy areas
for year in all_years:
    # Create a DataFrame with zeros for all EPG pairs
    merged_matrix = pd.DataFrame(0, index=all_epgs, columns=all_epgs)
    count_matrix = pd.DataFrame(0, index=all_epgs, columns=all_epgs)
    
    # Add similarity scores from each policy area
    for policy_area in similarity_matrices:
        if year in similarity_matrices[policy_area]:
            policy_matrix = similarity_matrices[policy_area][year]
            for epg1 in policy_matrix.index:
                for epg2 in policy_matrix.columns:
                    if epg1 in merged_matrix.index and epg2 in merged_matrix.columns:
                        merged_matrix.loc[epg1, epg2] += policy_matrix.loc[epg1, epg2]
                        count_matrix.loc[epg1, epg2] += 1
    
    # Average the similarity scores
    for epg1 in merged_matrix.index:
        for epg2 in merged_matrix.columns:
            if count_matrix.loc[epg1, epg2] > 0:
                merged_matrix.loc[epg1, epg2] /= count_matrix.loc[epg1, epg2]
            else:
                # No data for this pair, set to 0 if different EPGs, 1 if same EPG
                merged_matrix.loc[epg1, epg2] = 1.0 if epg1 == epg2 else 0.0
    
    # Store the merged matrix
    merged_similarity_matrices[year] = merged_matrix

# Get years with enough data
years = [year for year in all_years if len(merged_similarity_matrices[year]) >= 3]

# Make sure we have at least one valid year
if not years:
    raise ValueError("No years have enough EPGs for visualization")

# Get EPGs from the first year
epgs = list(merged_similarity_matrices[years[0]].index)

# Create color mapping for EPGs
epg_colors = {}
palette = Category10[10]
for i, epg in enumerate(epgs):
    epg_colors[epg] = palette[i % len(palette)]

# Function to perform dimensionality reduction on a similarity matrix
def get_coordinates(similarity_matrix, method='pca'):
    # Convert similarity to distance matrix
    distance_matrix = 1 - similarity_matrix
    
    # Convert to numpy array
    X = distance_matrix.values
    
    # Apply dimensionality reduction
    if method == 'pca':
        model = PCA(n_components=2)
        result = model.fit_transform(X)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Create DataFrame with results
    df_result = pd.DataFrame({
        'x': result[:, 0],
        'y': result[:, 1],
        'epg': distance_matrix.index,
        'color': [epg_colors.get(epg, '#000000') for epg in distance_matrix.index]
    })
    
    return df_result

# Function to align coordinates with reference using Procrustes analysis
def align_coordinates(target_df, reference_df):
    # Get common EPGs
    common_epgs = set(target_df['epg']).intersection(set(reference_df['epg']))
    
    if len(common_epgs) < 2:
        # Not enough common points to align
        return target_df
    
    # Extract coordinates for common EPGs
    target_coords = np.array([target_df[target_df['epg'] == epg][['x', 'y']].values[0] for epg in common_epgs])
    reference_coords = np.array([reference_df[reference_df['epg'] == epg][['x', 'y']].values[0] for epg in common_epgs])
    
    # Perform Procrustes analysis to align target to reference
    mtx1, mtx2, disparity = procrustes(reference_coords, target_coords)
    
    # Create transformation matrix (scale, rotation, reflection)
    scale = np.sqrt(np.sum(mtx1[0]**2)) / np.sqrt(np.sum(target_coords[0]**2))
    
    # Apply transformation to all points in target_df
    result_df = target_df.copy()
    coords = result_df[['x', 'y']].values
    
    # Scale and center (simplified Procrustes)
    coords_scaled = coords * scale
    
    # Get centroids
    target_centroid = np.mean(target_coords, axis=0)
    reference_centroid = np.mean(reference_coords, axis=0)
    
    # Translate
    coords_transformed = coords_scaled - target_centroid + reference_centroid
    
    # Update dataframe
    result_df['x'] = coords_transformed[:, 0]
    result_df['y'] = coords_transformed[:, 1]
    
    return result_df

# Compute coordinates for the first year (reference)
method = 'pca'  # PCA is more stable for small numbers of points
reference_data = get_coordinates(merged_similarity_matrices[years[0]], method=method)

# Compute and align coordinates for all years
aligned_data = {}
for year in years:
    try:
        # Compute initial coordinates
        year_data = get_coordinates(merged_similarity_matrices[year], method=method)
        
        # Align with reference
        if year == years[0]:
            aligned_data[year] = year_data  # Reference year doesn't need alignment
        else:
            aligned_data[year] = align_coordinates(year_data, reference_data)
    except Exception as e:
        print(f"Error processing year {year}: {e}")
        # Skip this year

# Make sure we have at least one valid year after processing
if not aligned_data:
    raise ValueError("No valid data after processing")

# Find the overall range of data across all years to set consistent plot boundaries
all_x = []
all_y = []
for year_data in aligned_data.values():
    all_x.extend(year_data['x'])
    all_y.extend(year_data['y'])

x_min, x_max = min(all_x), max(all_x)
y_min, y_max = min(all_y), max(all_y)

# Add padding (200% zoom - 50% padding on each side)
padding_x = (x_max - x_min) * 0.5
padding_y = (y_max - y_min) * 0.5
x_range = (x_min - padding_x, x_max + padding_x)
y_range = (y_min - padding_y, y_max + padding_y)

# Create initial plot with first year
current_year = years[0]
init_data = aligned_data[current_year]

# Create ColumnDataSource
source = ColumnDataSource(init_data)

# Create the figure with fixed range
p = figure(width=800, height=600, 
           title=f'EPG Clustering - All Policy Areas Combined ({current_year})',
           tools="pan,wheel_zoom,box_zoom,reset,save",
           x_range=Range1d(x_range[0], x_range[1]),
           y_range=Range1d(y_range[0], y_range[1]))

# Add scatter points
p.circle('x', 'y', source=source, size=15, color='color', alpha=0.8, 
         line_color='black', line_width=1)

# Add labels
labels = LabelSet(x='x', y='y', text='epg', source=source,
                 text_font_size='10pt', text_color='black',
                 x_offset=5, y_offset=5)
p.add_layout(labels)

# Prepare data for JavaScript
js_data = {}
for year in aligned_data.keys():
    js_data[str(year)] = aligned_data[year].to_dict('list')

# Get sorted years that have data
valid_years = sorted(aligned_data.keys())

# Create a slider for years
year_slider = Slider(title="Year", start=0, end=len(valid_years)-1, value=0, step=1, width=600)

# Create play/pause button
play_button = Button(label="▶️ Play", button_type="success", width=100)

# JavaScript callback for slider
slider_callback = CustomJS(args=dict(source=source, p=p, years=valid_years, data=js_data), code="""
    // Get the selected year index
    const yearIndex = cb_obj.value;
    const year = years[yearIndex];
    
    // Update data from precomputed results
    const year_data = data[year];
    
    // Update the source data
    source.data['x'] = year_data['x'];
    source.data['y'] = year_data['y'];
    source.data['epg'] = year_data['epg'];
    source.data['color'] = year_data['color'];
    
    // Update title
    p.title.text = `EPG Clustering - All Policy Areas Combined (${year})`;
    
    // Trigger update
    source.change.emit();
""")

# Connect callback to slider
year_slider.js_on_change('value', slider_callback)

# Animation callback for play button
animation_callback = CustomJS(args=dict(slider=year_slider, button=play_button), code="""
    if (button.label === "▶️ Play") {
        // Start animation
        button.label = "⏸️ Pause";
        
        // Function to increment slider
        function animate_slider() {
            if (button.label === "⏸️ Pause") {
                let current = slider.value;
                let next = current + 1;
                
                // Loop back to beginning if at the end
                if (next > slider.end) {
                    next = slider.start;
                }
                
                // Update slider value (this will trigger the slider callback)
                slider.value = next;
                
                // Schedule next update
                window.setTimeout(animate_slider, 1000);  // 1 second interval
            }
        }
        
        // Start animation
        animate_slider();
    } else {
        // Pause animation
        button.label = "▶️ Play";
    }
""")

# Connect animation callback to play button
play_button.js_on_click(animation_callback)

# Create layout
layout = column(
    row(year_slider, play_button),
    p
)

# Save the visualization to an HTML file
save(layout)

print("Visualization saved to 'epg_clustering_all_policies_merged.html'. Open this file in a web browser to view the animation.")

/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_80649/1900812374.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9811908320876931' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_matrix.loc[epg1, epg2] += policy_matrix.loc[epg1, epg2]
/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_80649/1900812374.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.41345291479820634' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_matrix.loc[epg1, epg2] += policy_matrix.loc[epg1, epg2]
/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_80649/1900812374.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7746469446489526'

Visualization saved to 'epg_clustering_all_policies_merged.html'. Open this file in a web browser to view the animation.


/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_80649/1900812374.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9043799784714747' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_matrix.loc[epg1, epg2] += policy_matrix.loc[epg1, epg2]
/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_80649/1900812374.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9179732776497493' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_matrix.loc[epg1, epg2] += policy_matrix.loc[epg1, epg2]
/var/folders/6w/0k457k8x1_97tynsvrbjwdgc0000gn/T/ipykernel_80649/1900812374.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.915563344770738' h

## Interactive verison with selecte clustering and policy area

In [36]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial import procrustes
from bokeh.plotting import figure, save, output_file
from bokeh.models import (ColumnDataSource, LabelSet, Slider, CustomJS, Button, 
                         Range1d, Select, RadioButtonGroup)
from bokeh.layouts import column, row
from bokeh.palettes import Category10

# Set output to an HTML file
output_file("epg_clustering_interactive.html")

# Get all available policy areas
policy_areas = sorted(similarity_matrices.keys())

# Create a function that will generate aligned coordinates for a given policy area and method
def generate_coordinates(policy_area, method='pca'):
    # Get years for this policy area
    years = sorted(similarity_matrices[policy_area].keys())
    
    # Get EPGs from the first year
    epgs = list(similarity_matrices[policy_area][years[0]].index)
    
    # Create color mapping for EPGs
    epg_colors = {}
    palette = Category10[10]
    for i, epg in enumerate(epgs):
        epg_colors[epg] = palette[i % len(palette)]
    
    # Function to perform dimensionality reduction on a similarity matrix
    def get_coordinates(similarity_matrix):
        # Convert similarity to distance matrix
        distance_matrix = 1 - similarity_matrix
        
        # Convert to numpy array
        X = distance_matrix.values
        
        # Apply dimensionality reduction
        if method == 'pca':
            model = PCA(n_components=2)
            result = model.fit_transform(X)
        elif method == 'tsne':
            # Use lower perplexity for small datasets
            perplexity = min(5, max(1, len(X) // 2))
            model = TSNE(n_components=2, perplexity=perplexity, 
                        random_state=42, learning_rate='auto', init='pca')
            result = model.fit_transform(X)
        else:
            raise ValueError(f"Unknown method: {method}")
        
        # Create DataFrame with results
        df_result = pd.DataFrame({
            'x': result[:, 0],
            'y': result[:, 1],
            'epg': distance_matrix.index,
            'color': [epg_colors.get(epg, '#000000') for epg in distance_matrix.index]
        })
        
        return df_result
    
    # Function to align coordinates with reference using Procrustes analysis
    def align_coordinates(target_df, reference_df):
        # Get common EPGs
        common_epgs = set(target_df['epg']).intersection(set(reference_df['epg']))
        
        if len(common_epgs) < 2:
            # Not enough common points to align
            return target_df
        
        # Extract coordinates for common EPGs
        target_coords = np.array([target_df[target_df['epg'] == epg][['x', 'y']].values[0] for epg in common_epgs])
        reference_coords = np.array([reference_df[reference_df['epg'] == epg][['x', 'y']].values[0] for epg in common_epgs])
        
        # Perform Procrustes analysis to align target to reference
        mtx1, mtx2, disparity = procrustes(reference_coords, target_coords)
        
        # Create transformation matrix (scale, rotation, reflection)
        scale = np.sqrt(np.sum(mtx1[0]**2)) / np.sqrt(np.sum(target_coords[0]**2))
        
        # Apply transformation to all points in target_df
        result_df = target_df.copy()
        coords = result_df[['x', 'y']].values
        
        # Scale and center (simplified Procrustes)
        coords_scaled = coords * scale
        
        # Get centroids
        target_centroid = np.mean(target_coords, axis=0)
        reference_centroid = np.mean(reference_coords, axis=0)
        
        # Translate
        coords_transformed = coords_scaled - target_centroid + reference_centroid
        
        # Update dataframe
        result_df['x'] = coords_transformed[:, 0]
        result_df['y'] = coords_transformed[:, 1]
        
        return result_df
    
    # Compute coordinates for the first year (reference)
    reference_data = get_coordinates(similarity_matrices[policy_area][years[0]])
    
    # Compute and align coordinates for all years
    aligned_data = {}
    for year in years:
        # Compute initial coordinates
        year_data = get_coordinates(similarity_matrices[policy_area][year])
        
        # Align with reference
        if year == years[0]:
            aligned_data[year] = year_data  # Reference year doesn't need alignment
        else:
            aligned_data[year] = align_coordinates(year_data, reference_data)
    
    # Find the overall range of data across all years to set consistent plot boundaries
    all_x = []
    all_y = []
    for year_data in aligned_data.values():
        all_x.extend(year_data['x'])
        all_y.extend(year_data['y'])
    
    x_min, x_max = min(all_x), max(all_x)
    y_min, y_max = min(all_y), max(all_y)
    
    # Add padding (200% zoom - 50% padding on each side)
    padding_x = (x_max - x_min) * 0.5
    padding_y = (y_max - y_min) * 0.5
    x_range = (x_min - padding_x, x_max + padding_x)
    y_range = (y_min - padding_y, y_max + padding_y)
    
    # Prepare data for JavaScript
    js_data = {}
    for year in years:
        js_data[str(year)] = aligned_data[year].to_dict('list')
    
    return {
        'years': years,
        'data': js_data,
        'init_data': aligned_data[years[0]],
        'x_range': x_range,
        'y_range': y_range
    }

# Generate initial data for the first policy area using PCA
initial_policy = policy_areas[0]
initial_method = 'pca'
initial_data = generate_coordinates(initial_policy, initial_method)

# Create ColumnDataSource
source = ColumnDataSource(initial_data['init_data'])

# Create the figure with fixed range
p = figure(width=800, height=600, 
           title=f'EPG Clustering - {initial_policy} ({initial_data["years"][0]})',
           tools="pan,wheel_zoom,box_zoom,reset,save",
           x_range=Range1d(initial_data['x_range'][0], initial_data['x_range'][1]),
           y_range=Range1d(initial_data['y_range'][0], initial_data['y_range'][1]))

# Add scatter points
p.circle('x', 'y', source=source, size=15, color='color', alpha=0.8, 
         line_color='black', line_width=1)

# Add labels
labels = LabelSet(x='x', y='y', text='epg', source=source,
                 text_font_size='10pt', text_color='black',
                 x_offset=5, y_offset=5)
p.add_layout(labels)

# Create controls
# Policy area dropdown
policy_select = Select(title="Policy Area:", value=initial_policy, options=policy_areas, width=200)

# Clustering method selection
method_group = RadioButtonGroup(labels=["PCA", "t-SNE"], active=0, width=200)

# Create a slider for years
year_slider = Slider(title="Year", start=0, end=len(initial_data['years'])-1, value=0, step=1, width=400)

# Create play/pause button
play_button = Button(label="▶️ Play", button_type="success", width=100)

# Precompute data for all policy areas and methods
all_data = {}
for policy in policy_areas:
    all_data[policy] = {}
    for method in ['pca', 'tsne']:
        try:
            print(f"Computing {method} for {policy}...")
            all_data[policy][method] = generate_coordinates(policy, method)
        except Exception as e:
            print(f"Error computing {method} for {policy}: {e}")
            # Create empty placeholder
            all_data[policy][method] = {
                'years': [],
                'data': {},
                'init_data': pd.DataFrame(columns=['x', 'y', 'epg', 'color']),
                'x_range': (-1, 1),
                'y_range': (-1, 1)
            }

# JavaScript callback for policy area selection
policy_callback = CustomJS(args=dict(source=source, p=p, year_slider=year_slider, 
                                   method_group=method_group, all_data=all_data), code="""
    // Get the selected policy area
    const policy = cb_obj.value;
    
    // Get the currently selected method
    const methods = ['pca', 'tsne'];
    const method = methods[method_group.active];
    
    // Get data for this policy and method
    const policy_data = all_data[policy][method];
    const years = policy_data.years;
    
    // Update slider
    year_slider.start = 0;
    year_slider.end = years.length - 1;
    year_slider.value = 0;
    
    // Get first year data
    const year = years[0];
    const year_data = policy_data.data[year];
    
    // Update the plot range
    p.x_range.start = policy_data.x_range[0];
    p.x_range.end = policy_data.x_range[1];
    p.y_range.start = policy_data.y_range[0];
    p.y_range.end = policy_data.y_range[1];
    
    // Update the source data
    source.data['x'] = year_data['x'];
    source.data['y'] = year_data['y'];
    source.data['epg'] = year_data['epg'];
    source.data['color'] = year_data['color'];
    
    // Update title
    p.title.text = `EPG Clustering - ${policy} (${year})`;
    
    // Trigger update
    source.change.emit();
""")

# JavaScript callback for method selection
method_callback = CustomJS(args=dict(source=source, p=p, year_slider=year_slider, 
                                  policy_select=policy_select, all_data=all_data), code="""
    // Get the selected method
    const methods = ['pca', 'tsne'];
    const method = methods[cb_obj.active];
    
    // Get the currently selected policy
    const policy = policy_select.value;
    
    // Get data for this policy and method
    const policy_data = all_data[policy][method];
    const years = policy_data.years;
    
    // Update slider
    year_slider.start = 0;
    year_slider.end = years.length - 1;
    year_slider.value = 0;
    
    // Get first year data
    const year = years[0];
    const year_data = policy_data.data[year];
    
    // Update the plot range
    p.x_range.start = policy_data.x_range[0];
    p.x_range.end = policy_data.x_range[1];
    p.y_range.start = policy_data.y_range[0];
    p.y_range.end = policy_data.y_range[1];
    
    // Update the source data
    source.data['x'] = year_data['x'];
    source.data['y'] = year_data['y'];
    source.data['epg'] = year_data['epg'];
    source.data['color'] = year_data['color'];
    
    // Update title
    p.title.text = `EPG Clustering - ${policy} (${year})`;
    
    // Trigger update
    source.change.emit();
""")

# JavaScript callback for slider
slider_callback = CustomJS(args=dict(source=source, p=p, policy_select=policy_select, 
                                  method_group=method_group, all_data=all_data), code="""
    // Get the selected policy and method
    const policy = policy_select.value;
    const methods = ['pca', 'tsne'];
    const method = methods[method_group.active];
    
    // Get data for this policy and method
    const policy_data = all_data[policy][method];
    const years = policy_data.years;
    
    // Get the selected year index
    const yearIndex = cb_obj.value;
    const year = years[yearIndex];
    
    // Update data from precomputed results
    const year_data = policy_data.data[year];
    
    // Update the source data
    source.data['x'] = year_data['x'];
    source.data['y'] = year_data['y'];
    source.data['epg'] = year_data['epg'];
    source.data['color'] = year_data['color'];
    
    // Update title
    p.title.text = `EPG Clustering - ${policy} (${year})`;
    
    // Trigger update
    source.change.emit();
""")

# Animation callback for play button
animation_callback = CustomJS(args=dict(slider=year_slider, button=play_button), code="""
    if (button.label === "▶️ Play") {
        // Start animation
        button.label = "⏸️ Pause";
        
        // Function to increment slider
        function animate_slider() {
            if (button.label === "⏸️ Pause") {
                let current = slider.value;
                let next = current + 1;
                
                // Loop back to beginning if at the end
                if (next > slider.end) {
                    next = slider.start;
                }
                
                // Update slider value (this will trigger the slider callback)
                slider.value = next;
                
                // Schedule next update
                window.setTimeout(animate_slider, 1000);  // 1 second interval
            }
        }
        
        // Start animation
        animate_slider();
    } else {
        // Pause animation
        button.label = "▶️ Play";
    }
""")

# Connect callbacks
policy_select.js_on_change('value', policy_callback)
method_group.js_on_change('active', method_callback)
year_slider.js_on_change('value', slider_callback)
play_button.js_on_click(animation_callback)

# Create layout
controls = row(
    column(
        policy_select,
        method_group
    ),
    column(
        row(year_slider, play_button)
    )
)

layout = column(
    controls,
    p
)

# Save the visualization to an HTML file
save(layout)

print("Visualization saved to 'epg_clustering_interactive.html'. Open this file in a web browser to interact with the visualization.")

Computing pca for agriculture...
Computing tsne for agriculture...
Computing pca for budget...
Computing tsne for budget...
Computing pca for budgetary control...
Computing tsne for budgetary control...
Computing pca for civil liberties justice home affairs...
Computing tsne for civil liberties justice home affairs...
Computing pca for constitutional inter institutional affairs...
Computing tsne for constitutional inter institutional affairs...
Computing pca for culture education...
Computing tsne for culture education...
Computing pca for development...
Computing tsne for development...
Computing pca for economic monetary affairs...
Computing tsne for economic monetary affairs...
Computing pca for economics...
Computing tsne for economics...
Computing pca for employment social affairs...
Computing tsne for employment social affairs...
Computing pca for environment public health...
Computing tsne for environment public health...
Computing pca for fisheries...
Computing tsne for fisheri

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.decomposition import PCA
from scipy.spatial import procrustes
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist, squareform
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

# Create directory for story plots if it doesn't exist
os.makedirs("story_plots", exist_ok=True)

# Calculate polarization metrics
def calculate_polarization_metrics(similarity_matrices):
    """Calculate cluster distance as a polarization metric for each year and policy area."""
    metrics = {}
    
    for policy_area in similarity_matrices:
        metrics[policy_area] = {}
        
        for year in similarity_matrices[policy_area]:
            sim_matrix = similarity_matrices[policy_area][year]
            
            # Skip if we don't have enough EPGs
            if len(sim_matrix) < 3:
                continue
                
            # Convert similarity to distance
            distance_matrix = 1 - sim_matrix
            
            # Calculate average distance (higher = more polarized)
            avg_distance = np.mean(distance_matrix.values[~np.eye(len(distance_matrix), dtype=bool)])
            
            # Calculate variance of distances (higher = more uneven polarization)
            var_distance = np.var(distance_matrix.values[~np.eye(len(distance_matrix), dtype=bool)])
            
            # Store metrics
            metrics[policy_area][year] = {
                'avg_distance': avg_distance,
                'var_distance': var_distance,
                'num_epgs': len(sim_matrix)
            }
    
    return metrics

# ----------------------
# PART 1: Overview Heatmap
# ----------------------

def create_overview_heatmap(similarity_matrices):
    """Create an aggregate heatmap of EPG similarities across all years and policy areas."""
    
    # Get all unique EPGs
    all_epgs = set()
    for policy_area in similarity_matrices:
        for year in similarity_matrices[policy_area]:
            matrix = similarity_matrices[policy_area][year]
            all_epgs.update(matrix.index)
    
    all_epgs = sorted(list(all_epgs))
    
    # Create an aggregate similarity matrix
    aggregate_matrix = pd.DataFrame(0.0, index=all_epgs, columns=all_epgs)
    count_matrix = pd.DataFrame(0, index=all_epgs, columns=all_epgs)
    
    # Sum all similarity matrices
    for policy_area in similarity_matrices:
        for year in similarity_matrices[policy_area]:
            matrix = similarity_matrices[policy_area][year]
            for i in matrix.index:
                for j in matrix.columns:
                    aggregate_matrix.loc[i, j] += matrix.loc[i, j]
                    count_matrix.loc[i, j] += 1
    
    # Average the similarities
    for i in aggregate_matrix.index:
        for j in aggregate_matrix.columns:
            if count_matrix.loc[i, j] > 0:
                aggregate_matrix.loc[i, j] /= count_matrix.loc[i, j]
            else:
                # If no data, set to NaN (will be shown as white/missing in heatmap)
                aggregate_matrix.loc[i, j] = np.nan
    
    # Set diagonal to 1 (self-similarity)
    for i in aggregate_matrix.index:
        aggregate_matrix.loc[i, i] = 1.0
    
    # Create the heatmap figure
    plt.figure(figsize=(12, 10))
    
    # Use a diverging colormap (blue to red)
    cmap = sns.diverging_palette(240, 10, as_cmap=True)
    
    # Create the heatmap
    sns.heatmap(aggregate_matrix, cmap=cmap, center=0.5,
                square=True, linewidths=.5, cbar_kws={"shrink": .8},
                vmin=0, vmax=1)
    
    plt.title('Overview: Average EPG Voting Similarity Across All Years and Policy Areas', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/1_overview_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return aggregate_matrix

# ----------------------
# PART 2: PCA Visualization
# ----------------------

def create_pca_visualization(aggregate_matrix):
    """Create a PCA visualization of EPG positions based on aggregate similarity."""
    
    # Clean the matrix (replace NaN with 0)
    clean_matrix = aggregate_matrix.fillna(0)
    
    # Convert similarity to distance
    distance_matrix = 1 - clean_matrix
    
    # Apply PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(distance_matrix)
    
    # Create a DataFrame for the results
    pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'], index=distance_matrix.index)
    
    # Organize EPGs into known political groups (rough approximation)
    # You'll need to adapt this based on your actual data
    political_spectrum = {
        'Left': ['GUE/NGL', 'G/EFA', 'LEFT'], 
        'Center-Left': ['S&D', 'SOC'],
        'Center': ['ALDE', 'RENEW'],
        'Center-Right': ['EPP', 'PPE', 'ECR'],
        'Right': ['ID', 'EFD', 'ENF', 'NI', 'EFDD', 'IND/DEM']
    }
    
    # Define colors for the political spectrum
    colors = {
        'Left': 'darkred',
        'Center-Left': 'tomato',
        'Center': 'purple',
        'Center-Right': 'royalblue',
        'Right': 'darkblue'
    }
    
    # Create the PCA visualization
    plt.figure(figsize=(10, 8))
    
    # Plot each EPG
    for epg in pca_df.index:
        # Find which group this EPG belongs to
        group = next((g for g, epgs in political_spectrum.items() if epg in epgs), 'Other')
        color = colors.get(group, 'gray')
        
        plt.scatter(pca_df.loc[epg, 'PC1'], pca_df.loc[epg, 'PC2'], 
                   color=color, s=100, alpha=0.7, edgecolors='black')
        plt.text(pca_df.loc[epg, 'PC1']+0.01, pca_df.loc[epg, 'PC2']+0.01, 
                epg, fontsize=12)
    
    # Add a legend
    for group, color in colors.items():
        plt.scatter([], [], color=color, label=group, s=100, alpha=0.7, edgecolors='black')
    plt.legend(title='Political Spectrum', fontsize=12)
    
    # Add labels and title
    explained_var = pca.explained_variance_ratio_
    plt.xlabel(f'Principal Component 1 ({explained_var[0]:.2%} variance)', fontsize=12)
    plt.ylabel(f'Principal Component 2 ({explained_var[1]:.2%} variance)', fontsize=12)
    plt.title('PCA: EPG Positioning Based on Voting Similarities', fontsize=14)
    
    # Add grid lines
    plt.grid(True, alpha=0.3)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/2_pca_visualization.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return pca_df

# ----------------------
# PART 3: Time Series Analysis
# ----------------------

def create_polarization_time_series(similarity_matrices, polarization_metrics):
    """Create time series plots of polarization metrics."""
    
    # Create a time series of metrics for each policy area
    time_series_data = {}
    
    for policy_area in polarization_metrics:
        years = sorted(polarization_metrics[policy_area].keys())
        
        avg_distances = [polarization_metrics[policy_area][year]['avg_distance'] 
                        if year in polarization_metrics[policy_area] else None 
                        for year in years]
        
        var_distances = [polarization_metrics[policy_area][year]['var_distance'] 
                        if year in polarization_metrics[policy_area] else None 
                        for year in years]
        
        num_epgs = [polarization_metrics[policy_area][year]['num_epgs'] 
                   if year in polarization_metrics[policy_area] else None 
                   for year in years]
        
        time_series_data[policy_area] = {
            'years': years,
            'avg_distance': avg_distances,
            'var_distance': var_distances,
            'num_epgs': num_epgs
        }
    
    # Find policy areas with most consistent data
    complete_policy_areas = []
    for policy_area, data in time_series_data.items():
        non_null_values = [x for x in data['avg_distance'] if x is not None]
        if len(non_null_values) > 5:  # At least 5 years of data
            complete_policy_areas.append(policy_area)
    
    # Calculate average polarization across all policy areas by year
    all_years = set()
    for policy_area in polarization_metrics:
        all_years.update(polarization_metrics[policy_area].keys())
    all_years = sorted(all_years)
    
    avg_polarization_by_year = {}
    for year in all_years:
        values = []
        for policy_area in polarization_metrics:
            if year in polarization_metrics[policy_area]:
                values.append(polarization_metrics[policy_area][year]['avg_distance'])
        
        if values:
            avg_polarization_by_year[year] = np.mean(values)
    
    # Create time series plot for average polarization
    plt.figure(figsize=(10, 6))
    
    years = sorted(avg_polarization_by_year.keys())
    values = [avg_polarization_by_year[year] for year in years]
    
    plt.plot(years, values, 'o-', color='black', linewidth=2.5, label='Average Across All Areas')
    
    # Select a few interesting policy areas to highlight
    interesting_areas = []
    for policy_area in complete_policy_areas:
        data = time_series_data[policy_area]
        non_null_distances = [d for d in data['avg_distance'] if d is not None]
        if non_null_distances:
            variance = np.var(non_null_distances)
            trend = np.polyfit([i for i, d in enumerate(data['avg_distance']) if d is not None], 
                               [d for d in data['avg_distance'] if d is not None], 1)[0]
            interesting_areas.append((policy_area, variance, trend))
    
    # Sort by variance (most varying first)
    interesting_areas.sort(key=lambda x: x[1], reverse=True)
    
    # Plot top 5 most interesting policy areas
    colors = plt.cm.Set2(np.linspace(0, 1, 5))
    for i, (policy_area, _, _) in enumerate(interesting_areas[:5]):
        data = time_series_data[policy_area]
        valid_years = [year for i, year in enumerate(data['years']) if data['avg_distance'][i] is not None]
        valid_values = [val for val in data['avg_distance'] if val is not None]
        
        if valid_years and valid_values:
            plt.plot(valid_years, valid_values, 'o-', color=colors[i], linewidth=1.5, label=policy_area)
    
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Polarization (Average Distance)', fontsize=12)
    plt.title('Polarization Trends Over Time by Policy Area', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Policy Area', fontsize=10)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/3_polarization_time_series.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return time_series_data, avg_polarization_by_year, interesting_areas

# ----------------------
# PART 4: Policy Area Ranking
# ----------------------

def create_policy_area_ranking(polarization_metrics, avg_polarization_by_year):
    """Create a visualization of policy areas ranked by polarization over time."""
    
    # Get all years and policy areas
    all_years = sorted(set(y for pa in polarization_metrics.values() for y in pa.keys()))
    policy_areas = list(polarization_metrics.keys())
    
    # Create a matrix of polarization values [policy_areas x years]
    polarization_matrix = pd.DataFrame(index=policy_areas, columns=all_years)
    
    for policy_area in policy_areas:
        for year in all_years:
            if year in polarization_metrics[policy_area]:
                polarization_matrix.loc[policy_area, year] = polarization_metrics[policy_area][year]['avg_distance']
    
    # Calculate overall polarization for each policy area (average over years)
    avg_polarization = polarization_matrix.mean(axis=1).dropna()
    
    # Sort policy areas by average polarization
    sorted_areas = avg_polarization.sort_values(ascending=False)
    
    # Take top and bottom 10 policy areas
    top_areas = sorted_areas.head(10)
    bottom_areas = sorted_areas.tail(10)
    
    # Create a figure with two subplots
    fig, axes = plt.subplots(2, 1, figsize=(12, 14), sharex=True)
    
    # Plot top 10 most polarized areas
    top_areas.plot(kind='barh', ax=axes[0], color='tomato')
    axes[0].set_title('Top 10 Most Polarized Policy Areas', fontsize=14)
    axes[0].set_xlabel('Average Polarization', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Plot bottom 10 least polarized areas
    bottom_areas.plot(kind='barh', ax=axes[1], color='skyblue')
    axes[1].set_title('Top 10 Least Polarized Policy Areas', fontsize=14)
    axes[1].set_xlabel('Average Polarization', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    # Adjust layout
    plt.tight_layout()
    plt.savefig('story_plots/4_policy_area_ranking.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a heatmap showing polarization for each policy area over time
    # Select top 15 most interesting areas (with highest variance)
    policy_var = polarization_matrix.var(axis=1).dropna()
    interesting_areas = policy_var.sort_values(ascending=False).head(15).index.tolist()
    
    # Create a filtered matrix with just these areas
    filtered_matrix = polarization_matrix.loc[interesting_areas].copy()
    
    # Convert all values to float and replace NaN with a clear indicator for plotting
    for col in filtered_matrix.columns:
        filtered_matrix[col] = filtered_matrix[col].astype(float)
    
    # Create heatmap using a manual approach to avoid the dtype error
    plt.figure(figsize=(12, 8))
    
    # Create a masked array for the heatmap
    masked_data = np.ma.masked_invalid(filtered_matrix.values)
    
    # Create heatmap manually
    im = plt.imshow(masked_data, cmap='RdBu_r', aspect='auto', interpolation='nearest')
    
    # Add color bar
    cbar = plt.colorbar(im, label='Polarization')
    
    # Set x and y ticks
    plt.yticks(range(len(filtered_matrix.index)), filtered_matrix.index)
    plt.xticks(range(len(filtered_matrix.columns)), 
              filtered_matrix.columns, rotation=45, ha='right')
    
    plt.title('Polarization Trends by Policy Area Over Time', fontsize=14)
    plt.ylabel('Policy Area', fontsize=12)
    plt.xlabel('Year', fontsize=12)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/5_policy_area_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return sorted_areas, filtered_matrix

# Main function to generate all visualizations
def generate_all_visualizations(similarity_matrices):
    """Generate all visualizations for the EPG polarization story."""
    
    # Calculate polarization metrics
    polarization_metrics = calculate_polarization_metrics(similarity_matrices)
    
    # Part 1: Overview Heatmap
    print("Creating overview heatmap...")
    aggregate_matrix = create_overview_heatmap(similarity_matrices)
    
    # Part 2: PCA Visualization
    print("Creating PCA visualization...")
    pca_df = create_pca_visualization(aggregate_matrix)
    
    # Part 3: Time Series Analysis
    print("Creating polarization time series...")
    time_series_data, avg_polarization_by_year, interesting_areas = create_polarization_time_series(
        similarity_matrices, polarization_metrics
    )
    
    # Part 4: Policy Area Ranking
    print("Creating policy area ranking...")
    sorted_areas, filtered_matrix = create_policy_area_ranking(
        polarization_metrics, avg_polarization_by_year
    )
    
    # Create a comprehensive story document
    story_text = """
# European Parliament Voting Patterns: A Story of Polarization

This visual narrative explores how voting patterns in the European Parliament have evolved over time, with a focus on political polarization across different policy areas.

## Part 1: The Overview

The heatmap provides a broad overview of voting similarities between different European Parliament Groups (EPGs) across all years and policy areas. Bright red areas indicate high similarity (groups voting together), while blue areas show dissimilarity (groups voting against each other).

![Overview Heatmap](story_plots/1_overview_heatmap.png)

This visualization reveals the fundamental structure of EPG voting patterns. We can observe clear clustering of ideologically similar groups, with a general left-right spectrum visible. The progressive-left groups (GUE/NGL, Greens) show high internal similarity, as do the center-right and conservative groups (EPP, ECR). This confirms the expected ideological divisions in the Parliament.

## Part 2: EPG Positioning in Two Dimensions

Using Principal Component Analysis (PCA), we can visualize the relative positioning of EPGs based on their voting similarities:

![PCA Visualization](story_plots/2_pca_visualization.png)

The PCA plot reveals how EPGs are positioned relative to each other based on their voting behavior. The first principal component (x-axis) largely corresponds to the traditional left-right political spectrum, while the second component (y-axis) may represent other dimensions of political division such as attitudes toward European integration or social issues.

We can observe distinct clusters forming around traditional political families: the progressive left, social democrats, liberals, conservatives, and right-wing groups. The distance between groups in this visualization represents their voting dissimilarity.

## Part 3: Polarization Trends Over Time

How has polarization in the European Parliament evolved over time? The following time series shows the average distance between EPGs (our polarization metric) for selected policy areas:

![Polarization Time Series](story_plots/3_polarization_time_series.png)

The black line represents the average polarization across all policy areas, providing a reference point. We can observe that certain policy areas show significantly different polarization trends than others. Some areas have become increasingly polarized over time, while others have shown convergence.

Of particular note are the following trends:
- The overall polarization (black line) shows a general [increasing/decreasing/stable] trend over the years.
- [Specific policy area] shows the most dramatic increase in polarization.
- [Specific policy area] has become less polarized over time, suggesting growing consensus.
- Periods of [political events, elections, crises] appear to coincide with [increases/decreases] in polarization.

## Part 4: Which Policy Areas Are Most Polarizing?

Not all policy areas generate the same level of political division. The following visualizations rank policy areas by their average polarization:

![Policy Area Ranking](story_plots/4_policy_area_ranking.png)

The most polarized policy areas tend to relate to [specific types of policies: e.g., migration, economic policy, social issues], which aligns with our understanding of the most contentious issues in European politics.

In contrast, the least polarized areas are generally [specific types of policies: e.g., technical standards, research funding, infrastructure], where there tends to be more technocratic consensus.

Finally, we can examine how polarization in different policy areas has evolved over time:

![Policy Area Heatmap](story_plots/5_policy_area_heatmap.png)

This heatmap reveals interesting patterns of political conflict over time. We can see that:
- Some policy areas (e.g., [specific examples]) have become increasingly polarized.
- Others (e.g., [specific examples]) show decreasing polarization.
- Certain years show higher overall polarization, potentially corresponding to [specific political events or crises].

## Conclusion

Our visual analysis of European Parliament voting patterns reveals a complex landscape of political alignment and polarization. While the traditional left-right spectrum remains evident in voting behavior, polarization varies significantly across policy areas and time periods.

The findings suggest that [summarize key insights about polarization trends, their potential causes, and implications for European politics].

This analysis provides a foundation for understanding political dynamics in the European Parliament and how they have evolved over time. Future research could explore the causes of polarization in specific policy areas and the impact of external events on parliamentary voting patterns.
"""
    
    # Save the story text
    with open('story_plots/polarization_story.md', 'w') as f:
        f.write(story_text)
    
    print("All visualizations and story document created in the 'story_plots' folder.")

# You would call generate_all_visualizations(similarity_matrices) to create all plots and story

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.decomposition import PCA
from scipy.spatial import procrustes
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist, squareform
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec
from bokeh.plotting import figure, save, output_file
from bokeh.models import (ColumnDataSource, HoverTool, CustomJS, Slider, Button, 
                         Range1d, LinearColorMapper, ColorBar, BasicTicker, 
                         MultiSelect, CheckboxGroup, Legend, LegendItem)
from bokeh.layouts import column, row, gridplot
from bokeh.palettes import RdBu11, Spectral11, Category20, viridis
from bokeh.transform import linear_cmap
from scipy.stats import linregress

# Create directory for story plots if it doesn't exist
os.makedirs("story_plots", exist_ok=True)

# Calculate polarization metrics
def calculate_polarization_metrics(similarity_matrices):
    """Calculate cluster distance as a polarization metric for each year and policy area."""
    metrics = {}
    
    for policy_area in similarity_matrices:
        metrics[policy_area] = {}
        
        for year in similarity_matrices[policy_area]:
            sim_matrix = similarity_matrices[policy_area][year]
            
            # Skip if we don't have enough EPGs
            if len(sim_matrix) < 3:
                continue
                
            # Convert similarity to distance
            distance_matrix = 1 - sim_matrix
            
            # Calculate average distance (higher = more polarized)
            avg_distance = np.mean(distance_matrix.values[~np.eye(len(distance_matrix), dtype=bool)])
            
            # Calculate variance of distances (higher = more uneven polarization)
            var_distance = np.var(distance_matrix.values[~np.eye(len(distance_matrix), dtype=bool)])
            
            # Calculate modularity-like measure (higher = more distinct communities)
            # This uses variance/mean as a simple proxy for community structure
            distances_flat = distance_matrix.values[~np.eye(len(distance_matrix), dtype=bool)].flatten()
            if np.mean(distances_flat) > 0:
                modularity = np.var(distances_flat) / np.mean(distances_flat)
            else:
                modularity = 0
            
            # Calculate cohesion within ideological groups
            # Define ideological groups (adapt based on your data)
            ideological_groups = {
                'Left': ['GUE/NGL', 'LEFT', 'THE LEFT', 'GUE-NGL'], 
                'Greens': ['G/EFA', 'Greens/EFA', 'The Greens', 'GREENS'],
                'Social Democrats': ['S&D', 'SOC', 'PES', 'SD', 'PSE'],
                'Liberals': ['ALDE', 'RENEW', 'RE', 'ALDE/ADLE'],
                'Christian Democrats': ['EPP', 'PPE', 'PPE-DE', 'EPP-ED'],
                'Conservatives': ['ECR', 'UEN'],
                'Right-wing': ['ID', 'EFD', 'ENF', 'EFDD', 'IND/DEM', 'ITS'],
                'Regionalists': ['EFA', 'REG', 'EDA'],
                'Non-affiliated': ['NI']
            }
            
            # Calculate in-group vs out-group distances
            in_group_distances = []
            out_group_distances = []
            
            for i, epg_i in enumerate(distance_matrix.index):
                for j, epg_j in enumerate(distance_matrix.index):
                    if i != j:  # Skip self-comparisons
                        distance = distance_matrix.iloc[i, j]
                        
                        # Check if both EPGs are in the same ideological group
                        same_group = False
                        for group, members in ideological_groups.items():
                            if epg_i in members and epg_j in members:
                                same_group = True
                                break
                        
                        if same_group:
                            in_group_distances.append(distance)
                        else:
                            out_group_distances.append(distance)
            
            # Calculate ideological cohesion (lower = more cohesive within ideological groups)
            if in_group_distances and out_group_distances:
                ideological_cohesion = np.mean(in_group_distances) / np.mean(out_group_distances)
            else:
                ideological_cohesion = np.nan
            
            # Store metrics
            metrics[policy_area][year] = {
                'avg_distance': avg_distance,
                'var_distance': var_distance,
                'modularity': modularity,
                'ideological_cohesion': ideological_cohesion,
                'num_epgs': len(sim_matrix)
            }
    
    return metrics

# ----------------------
# PART 1: Overview Heatmap with Enhanced Colors
# ----------------------

def create_overview_heatmap(similarity_matrices):
    """Create an aggregate heatmap of EPG similarities across all years and policy areas."""
    
    # Get all unique EPGs
    all_epgs = set()
    for policy_area in similarity_matrices:
        for year in similarity_matrices[policy_area]:
            matrix = similarity_matrices[policy_area][year]
            all_epgs.update(matrix.index)
    
    all_epgs = sorted(list(all_epgs))
    
    # Create an aggregate similarity matrix
    aggregate_matrix = pd.DataFrame(0.0, index=all_epgs, columns=all_epgs)
    count_matrix = pd.DataFrame(0, index=all_epgs, columns=all_epgs)
    
    # Sum all similarity matrices
    for policy_area in similarity_matrices:
        for year in similarity_matrices[policy_area]:
            matrix = similarity_matrices[policy_area][year]
            for i in matrix.index:
                for j in matrix.columns:
                    aggregate_matrix.loc[i, j] += matrix.loc[i, j]
                    count_matrix.loc[i, j] += 1
    
    # Average the similarities
    for i in aggregate_matrix.index:
        for j in aggregate_matrix.columns:
            if count_matrix.loc[i, j] > 0:
                aggregate_matrix.loc[i, j] /= count_matrix.loc[i, j]
            else:
                # If no data, set to NaN (will be shown as white/missing in heatmap)
                aggregate_matrix.loc[i, j] = np.nan
    
    # Set diagonal to 1 (self-similarity)
    for i in aggregate_matrix.index:
        aggregate_matrix.loc[i, i] = 1.0
    
    # Create the heatmap figure
    plt.figure(figsize=(12, 10))
    
    # Calculate the center point dynamically to enhance color contrast
    sim_values = aggregate_matrix.values.flatten()
    sim_values = sim_values[~np.isnan(sim_values)]
    median_sim = np.median(sim_values)
    
    # Use a diverging colormap with enhanced contrast
    cmap = sns.diverging_palette(220, 20, as_cmap=True)  # Blue to red with more contrast
    
    # Create the heatmap with enhanced dynamic range
    sns.heatmap(aggregate_matrix, cmap=cmap, center=median_sim,
                square=True, linewidths=.5, cbar_kws={"shrink": .8},
                vmin=max(0, median_sim - 0.3), vmax=min(1, median_sim + 0.3))  # Adjust range for more color variation
    
    plt.title('Overview: Average EPG Voting Similarity Across All Years and Policy Areas', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/1_overview_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return aggregate_matrix

# ----------------------
# PART 2: Enhanced PCA Visualization with Correct Political Spectrum
# ----------------------

def create_pca_visualization(aggregate_matrix):
    """Create a PCA visualization of EPG positions based on aggregate similarity."""
    
    # Clean the matrix (replace NaN with 0)
    clean_matrix = aggregate_matrix.fillna(0)
    
    # Convert similarity to distance
    distance_matrix = 1 - clean_matrix
    
    # Apply PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(distance_matrix)
    
    # Create a DataFrame for the results
    pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'], index=distance_matrix.index)
    
    # Organize EPGs into known political groups (comprehensive mapping)
    political_spectrum = {
        'Left': ['GUE/NGL', 'LEFT', 'THE LEFT', 'GUE-NGL'],
        'Greens': ['G/EFA', 'Greens/EFA', 'The Greens', 'GREENS'],
        'Social Democrats': ['S&D', 'SOC', 'PES', 'SD', 'PSE'],
        'Liberals': ['ALDE', 'RENEW', 'RE', 'ALDE/ADLE'],
        'Christian Democrats': ['EPP', 'PPE', 'PPE-DE', 'EPP-ED'],
        'Conservatives': ['ECR', 'UEN'],
        'Right-wing': ['ID', 'EFD', 'ENF', 'EFDD', 'IND/DEM', 'ITS'],
        'Regionalists': ['EFA', 'REG', 'EDA'],
        'Non-affiliated': ['NI']
    }
    
    # Define colors for the political spectrum (enhanced palette)
    colors = {
        'Left': '#d62728',  # Dark red
        'Greens': '#2ca02c',  # Green
        'Social Democrats': '#ff7f0e',  # Orange
        'Liberals': '#ffff00',  # Yellow
        'Christian Democrats': '#1f77b4',  # Blue
        'Conservatives': '#9467bd',  # Purple
        'Right-wing': '#8c564b',  # Brown
        'Regionalists': '#e377c2',  # Pink
        'Non-affiliated': '#7f7f7f'   # Grey
    }
    
    # Create the PCA visualization
    plt.figure(figsize=(12, 10))
    
    # Create a dictionary to store points for legend (to avoid duplicates)
    legend_handles = {}
    
    # Check each EPG and assign to a political group
    epg_groups = {}
    for epg in pca_df.index:
        assigned = False
        for group, members in political_spectrum.items():
            if any(member.lower() in epg.lower() or epg.lower() in member.lower() for member in members):
                epg_groups[epg] = group
                assigned = True
                break
        if not assigned:
            # Try a more flexible matching for any remaining groups
            for group, members in political_spectrum.items():
                if any(member.split('/')[0] in epg for member in members if '/' in member):
                    epg_groups[epg] = group
                    assigned = True
                    break
        if not assigned:
            # Last resort: check for common abbreviations
            if 'GUE' in epg or 'LEFT' in epg.upper():
                epg_groups[epg] = 'Left'
            elif 'GREEN' in epg.upper() or 'G/E' in epg:
                epg_groups[epg] = 'Greens'
            elif 'S&D' in epg or 'SOC' in epg.upper() or 'SD' in epg.upper():
                epg_groups[epg] = 'Social Democrats'
            elif 'ALDE' in epg or 'LIB' in epg.upper() or 'RENEW' in epg.upper():
                epg_groups[epg] = 'Liberals'
            elif 'EPP' in epg or 'PPE' in epg:
                epg_groups[epg] = 'Christian Democrats'
            elif 'ECR' in epg:
                epg_groups[epg] = 'Conservatives'
            elif 'EFD' in epg or 'ID' in epg or 'ENF' in epg:
                epg_groups[epg] = 'Right-wing'
            elif 'REG' in epg.upper() or 'EFA' in epg:
                epg_groups[epg] = 'Regionalists'
            elif 'NI' in epg:
                epg_groups[epg] = 'Non-affiliated'
            else:
                # If still can't determine, just print a warning and set to 'Other'
                print(f"Warning: Could not assign {epg} to a political group")
                epg_groups[epg] = 'Other'
    
    # Plot each EPG
    for epg in pca_df.index:
        group = epg_groups.get(epg, 'Other')
        color = colors.get(group, 'gray')
        
        # Plot the point
        plt.scatter(pca_df.loc[epg, 'PC1'], pca_df.loc[epg, 'PC2'], 
                   color=color, s=120, alpha=0.8, edgecolors='black')
        
        # Add text label for the EPG
        plt.text(pca_df.loc[epg, 'PC1']+0.02, pca_df.loc[epg, 'PC2']+0.02, 
                epg, fontsize=12, weight='bold')
        
        # Add to legend handles
        if group not in legend_handles:
            legend_handles[group] = plt.Line2D([0], [0], marker='o', color='w', 
                                             markerfacecolor=color, markersize=12, 
                                             label=group)
    
    # Add a legend with political groups
    plt.legend(handles=list(legend_handles.values()), 
              title='Political Groups', fontsize=12, 
              title_fontsize=14, loc='best')
    
    # Add labels and title
    explained_var = pca.explained_variance_ratio_
    plt.xlabel(f'Principal Component 1 ({explained_var[0]:.2%} variance)', fontsize=14)
    plt.ylabel(f'Principal Component 2 ({explained_var[1]:.2%} variance)', fontsize=14)
    plt.title('PCA: European Parliament Groups Positioned by Voting Patterns', fontsize=16)
    
    # Add grid lines and enhance visualization
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Add horizontal and vertical lines at origin for reference
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/2_pca_visualization.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return pca_df

# ----------------------
# PART 3: Interactive Time Series with Multiple Selection
# ----------------------

def create_interactive_polarization_time_series(similarity_matrices, polarization_metrics):
    """Create interactive time series plots of polarization metrics with Bokeh."""
    
    # Create a time series of metrics for each policy area
    time_series_data = {}
    
    for policy_area in polarization_metrics:
        years = sorted(polarization_metrics[policy_area].keys())
        
        avg_distances = [polarization_metrics[policy_area][year]['avg_distance'] 
                        if year in polarization_metrics[policy_area] else None 
                        for year in years]
        
        var_distances = [polarization_metrics[policy_area][year]['var_distance'] 
                        if year in polarization_metrics[policy_area] else None 
                        for year in years]
        
        modularity = [polarization_metrics[policy_area][year]['modularity'] 
                     if year in polarization_metrics[policy_area] else None 
                     for year in years]
        
        if 'ideological_cohesion' in next(iter(polarization_metrics[policy_area].values()), {}):
            cohesion = [polarization_metrics[policy_area][year]['ideological_cohesion'] 
                      if year in polarization_metrics[policy_area] and not np.isnan(polarization_metrics[policy_area][year]['ideological_cohesion']) 
                      else None 
                      for year in years]
        else:
            cohesion = [None] * len(years)
        
        num_epgs = [polarization_metrics[policy_area][year]['num_epgs'] 
                   if year in polarization_metrics[policy_area] else None 
                   for year in years]
        
        time_series_data[policy_area] = {
            'years': years,
            'avg_distance': avg_distances,
            'var_distance': var_distances,
            'modularity': modularity,
            'cohesion': cohesion,
            'num_epgs': num_epgs
        }
    
    # Filter out policy areas with too little data
    filtered_policy_areas = {}
    for policy_area, data in time_series_data.items():
        valid_years = [i for i, d in enumerate(data['avg_distance']) if d is not None]
        if len(valid_years) >= 5:  # At least 5 years of data
            filtered_policy_areas[policy_area] = {
                'years': [data['years'][i] for i in valid_years],
                'avg_distance': [data['avg_distance'][i] for i in valid_years],
                'var_distance': [data['var_distance'][i] for i in valid_years],
                'modularity': [data['modularity'][i] for i in valid_years] if any(data['modularity']) else None,
                'cohesion': [data['cohesion'][i] for i in valid_years] if any(data['cohesion']) else None,
                'num_epgs': [data['num_epgs'][i] for i in valid_years]
            }
    
    # Calculate average polarization across all policy areas by year
    all_years = set()
    for policy_area in polarization_metrics:
        all_years.update(polarization_metrics[policy_area].keys())
    all_years = sorted(all_years)
    
    avg_polarization_by_year = {}
    for year in all_years:
        values = []
        for policy_area in polarization_metrics:
            if year in polarization_metrics[policy_area]:
                values.append(polarization_metrics[policy_area][year]['avg_distance'])
        
        if values:
            avg_polarization_by_year[year] = np.mean(values)
    
    # Create an interactive Bokeh plot
    output_file('story_plots/3_interactive_polarization.html')
    
    # Setup initial data source for the average across all policy areas
    avg_source = ColumnDataSource(data=dict(
        years=list(avg_polarization_by_year.keys()),
        values=list(avg_polarization_by_year.values()),
        policy=['Average Across All Areas'] * len(avg_polarization_by_year)
    ))
    
    # Create the figure
    p = figure(width=900, height=600, 
              title='Polarization Trends Over Time by Policy Area',
              x_axis_label='Year', y_axis_label='Polarization (Average Distance)',
              tools="pan,wheel_zoom,box_zoom,reset,save,hover")
    
    # Configure hover tool
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
        ("Policy Area", "@policy"),
        ("Year", "@years"),
        ("Polarization", "@values{0.000}")
    ]
    
    # Plot the average line
    avg_line = p.line('years', 'values', source=avg_source, line_width=3, 
                     color='black', alpha=0.8, legend_label="Average Across All Areas")
    avg_circle = p.circle('years', 'values', source=avg_source, size=8, 
                         color='black', alpha=0.8)
    
    # Create multi select widget for policy areas
    sorted_areas = sorted(filtered_policy_areas.keys())
    
    # Select top 5 most varying policy areas as default selected
    policy_variance = []
    for policy in sorted_areas:
        data = filtered_policy_areas[policy]
        if len(data['avg_distance']) > 0:
            policy_variance.append((policy, np.var(data['avg_distance'])))
    
    policy_variance.sort(key=lambda x: x[1], reverse=True)
    default_selected = [p[0] for p in policy_variance[:5]]
    
    # Create a mapping of colors for each policy area
    color_palette = Category20[20]
    policy_colors = {policy: color_palette[i % 20] for i, policy in enumerate(sorted_areas)}
    
    # Create the multi-select widget
    policy_select = MultiSelect(title="Select Policy Areas to Compare:",
                              options=sorted_areas,
                              value=default_selected,
                              height=300,
                              width=300)
    
    # Create data sources for each policy area (initially empty)
    policy_sources = {}
    policy_lines = {}
    policy_circles = {}
    
    for policy in sorted_areas:
        data = filtered_policy_areas[policy]
        
        # Create source
        policy_sources[policy] = ColumnDataSource(data=dict(
            years=data['years'],
            values=data['avg_distance'],
            policy=[policy] * len(data['years']),
            visible=[policy in default_selected] * len(data['years'])
        ))
        
        # Create line and circle, set initial visibility
        visible = policy in default_selected
        
        policy_lines[policy] = p.line('years', 'values', source=policy_sources[policy],
                                    line_width=2, color=policy_colors[policy],
                                    alpha=0.8 if visible else 0,
                                    legend_label=policy if visible else "")
        
        policy_circles[policy] = p.circle('years', 'values', source=policy_sources[policy],
                                        size=6, color=policy_colors[policy],
                                        alpha=0.8 if visible else 0)
    
    # Add legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    
    # Create a callback for MultiSelect widget
    callback = CustomJS(args=dict(
        policy_sources=policy_sources,
        policy_lines=policy_lines,
        policy_circles=policy_circles,
        policy_colors=policy_colors,
        p=p), code="""
        // Get selected policies
        const selected_policies = cb_obj.value;
        
        // Update each policy line and circle visibility
        for (const policy in policy_sources) {
            const is_selected = selected_policies.includes(policy);
            const alpha = is_selected ? 0.8 : 0;
            
            // Update alpha for lines and circles
            policy_lines[policy].glyph.line_alpha = alpha;
            policy_circles[policy].glyph.fill_alpha = alpha;
            policy_circles[policy].glyph.line_alpha = alpha;
            
            // Update legend (this doesn't work perfectly in Bokeh callbacks)
            if (is_selected) {
                policy_lines[policy].legend_label = policy;
            } else {
                policy_lines[policy].legend_label = "";
            }
        }
        
        // This will trigger a redraw
        for (const policy in policy_sources) {
            policy_sources[policy].change.emit();
        }
    """)
    
    # Attach callback to widget
    policy_select.js_on_change('value', callback)
    
    # Layout
    layout = row(
        policy_select,
        p
    )
    
    # Save to file
    save(layout)
    
    # Also create a static version for the story
    plt.figure(figsize=(10, 6))
    
    # Plot average line
    years = list(avg_polarization_by_year.keys())
    values = list(avg_polarization_by_year.values())
    plt.plot(years, values, 'o-', color='black', linewidth=2.5, label='Average Across All Areas')
    
    # Plot a few interesting policy areas
    colors = plt.cm.tab10(np.linspace(0, 1, 5))
    for i, policy in enumerate(default_selected[:5]):
        data = filtered_policy_areas[policy]
        plt.plot(data['years'], data['avg_distance'], 'o-', 
                color=colors[i], linewidth=1.5, label=policy)
    
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Polarization (Average Distance)', fontsize=12)
    plt.title('Polarization Trends Over Time by Policy Area', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Policy Area', fontsize=10)
    
    # Save the static figure
    plt.tight_layout()
    plt.savefig('story_plots/3_polarization_time_series.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return filtered_policy_areas, avg_polarization_by_year

# ----------------------
# PART 4: Policy Area Ranking (Keep Same)
# ----------------------

def create_policy_area_ranking(polarization_metrics, avg_polarization_by_year):
    """Create a visualization of policy areas ranked by polarization over time."""
    
    # Get all years and policy areas
    all_years = sorted(set(y for pa in polarization_metrics.values() for y in pa.keys()))
    policy_areas = list(polarization_metrics.keys())
    
    # Create a matrix of polarization values [policy_areas x years]
    polarization_matrix = pd.DataFrame(index=policy_areas, columns=all_years)
    
    for policy_area in policy_areas:
        for year in all_years:
            if year in polarization_metrics[policy_area]:
                polarization_matrix.loc[policy_area, year] = polarization_metrics[policy_area][year]['avg_distance']
    
    # Calculate overall polarization for each policy area (average over years)
    avg_polarization = polarization_matrix.mean(axis=1).dropna()
    
    # Sort policy areas by average polarization
    sorted_areas = avg_polarization.sort_values(ascending=False)
    
    # Take top and bottom 10 policy areas
    top_areas = sorted_areas.head(10)
    bottom_areas = sorted_areas.tail(10)
    
    # Create a figure with two subplots
    fig, axes = plt.subplots(2, 1, figsize=(12, 14), sharex=True)
    
    # Plot top 10 most polarized areas
    top_areas.plot(kind='barh', ax=axes[0], color='tomato')
    axes[0].set_title('Top 10 Most Polarized Policy Areas', fontsize=14)
    axes[0].set_xlabel('Average Polarization', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Plot bottom 10 least polarized areas
    bottom_areas.plot(kind='barh', ax=axes[1], color='skyblue')
    axes[1].set_title('Top 10 Least Polarized Policy Areas', fontsize=14)
    axes[1].set_xlabel('Average Polarization', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    # Adjust layout
    plt.tight_layout()
    plt.savefig('story_plots/4_policy_area_ranking.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return sorted_areas, polarization_matrix

# ----------------------
# PART 5: Polarization Trends Visualization (Fixed)
# ----------------------

def create_polarization_patterns_visualization(polarization_metrics):
    """Create visualizations of polarization patterns showing clear trends over time."""
    
    # Extract trends in polarization over time for each policy area
    policy_trends = {}
    
    for policy in polarization_metrics:
        years = sorted(polarization_metrics[policy].keys())
        if len(years) >= 5:  # Only consider policies with enough data points
            values = [polarization_metrics[policy][year]['avg_distance'] for year in years]
            
            # Calculate linear trend
            if len(years) > 1:
                try:
                    slope, intercept, r_value, p_value, std_err = linregress(years, values)
                    policy_trends[policy] = {
                        'slope': slope,
                        'r_value': r_value,
                        'p_value': p_value,
                        'years': years,
                        'values': values
                    }
                except Exception as e:
                    print(f"Error calculating trend for {policy}: {e}")
    
    # Sort policies by trend strength (descending by absolute slope)
    sorted_policies = sorted(policy_trends.keys(), 
                            key=lambda p: abs(policy_trends[p]['slope']), 
                            reverse=True)
    
    # Select top 5 increasing and top 5 decreasing trends
    increasing = [p for p in sorted_policies if policy_trends[p]['slope'] > 0][:5]
    decreasing = [p for p in sorted_policies if policy_trends[p]['slope'] < 0][:5]
    
    # Create a figure showing diverging trends
    plt.figure(figsize=(12, 8))
    
    # Plot increasing trends
    plt.subplot(2, 1, 1)
    for i, policy in enumerate(increasing):
        data = policy_trends[policy]
        plt.plot(data['years'], data['values'], 'o-', 
                label=f"{policy} (r={data['r_value']:.2f})")
    
    plt.title('Policy Areas with Increasing Polarization', fontsize=14)
    plt.ylabel('Polarization', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=10)
    
    # Plot decreasing trends
    plt.subplot(2, 1, 2)
    for i, policy in enumerate(decreasing):
        data = policy_trends[policy]
        plt.plot(data['years'], data['values'], 'o-', 
                label=f"{policy} (r={data['r_value']:.2f})")
    
    plt.title('Policy Areas with Decreasing Polarization', fontsize=14)
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Polarization', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=10)
    
    plt.tight_layout()
    plt.savefig('story_plots/5_polarization_trends.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a second visualization: distribution of policy areas by trend
    slopes = [policy_trends[p]['slope'] for p in policy_trends]
    
    plt.figure(figsize=(10, 6))
    plt.hist(slopes, bins=20, alpha=0.7, color='steelblue', edgecolor='black')
    plt.axvline(x=0, color='r', linestyle='--', alpha=0.7)
    plt.xlabel('Polarization Trend (Slope)', fontsize=12)
    plt.ylabel('Number of Policy Areas', fontsize=12)
    plt.title('Distribution of Polarization Trends Across Policy Areas', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Add annotations
    mean_slope = np.mean(slopes)
    plt.axvline(x=mean_slope, color='g', linestyle='-', alpha=0.7)
    plt.text(mean_slope+0.001, plt.gca().get_ylim()[1]*0.9, 
            f'Mean: {mean_slope:.4f}', 
            color='g', fontsize=12)
    
    # Count increasing vs decreasing
    n_increasing = sum(s > 0 for s in slopes)
    n_decreasing = sum(s < 0 for s in slopes)
    percent_increasing = (n_increasing / len(slopes)) * 100
    
    plt.text(0.05, 0.95, f'Increasing: {n_increasing} ({percent_increasing:.1f}%)\nDecreasing: {n_decreasing} ({100-percent_increasing:.1f}%)', 
            transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
    
    plt.tight_layout()
    plt.savefig('story_plots/6_trend_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return policy_trends

# ----------------------
# PART 6: Interesting Metrics Visualization
# ----------------------

def create_interesting_metrics_visualization(polarization_metrics):
    """Create visualizations with interesting new metrics about political dynamics."""
    
    # Create time series for the ideological cohesion metric
    all_years = sorted(set(y for pa in polarization_metrics.values() for y in pa.keys()))
    
    # Get cohesion values for each policy area and year
    cohesion_data = {}
    for policy_area in polarization_metrics:
        cohesion_data[policy_area] = {}
        for year in all_years:
            if year in polarization_metrics[policy_area] and 'ideological_cohesion' in polarization_metrics[policy_area][year]:
                cohesion = polarization_metrics[policy_area][year]['ideological_cohesion']
                if not np.isnan(cohesion):
                    cohesion_data[policy_area][year] = cohesion
    
    # Find policy areas with sufficient data
    policy_with_data = {p: years for p, years in cohesion_data.items() if len(years) >= 5}
    
    if not policy_with_data:
        # If ideological cohesion data isn't available, return from the trends function
        return None
    
    # Calculate average cohesion across all policy areas
    avg_cohesion = {}
    for year in all_years:
        values = [cohesion_data[p][year] for p in cohesion_data if year in cohesion_data[p]]
        if values:
            avg_cohesion[year] = np.mean(values)
    
    # Calculate variance in cohesion across policy areas
    policy_cohesion_variance = {}
    for policy in policy_with_data:
        values = list(cohesion_data[policy].values())
        if values:
            policy_cohesion_variance[policy] = np.var(values)
    
    # Select top 5 policy areas with highest variance in cohesion
    top_varying = sorted(policy_cohesion_variance.items(), key=lambda x: x[1], reverse=True)[:5]
    
    # Create a figure for ideological cohesion over time
    plt.figure(figsize=(10, 6))
    
    # Plot average cohesion
    years = sorted(avg_cohesion.keys())
    values = [avg_cohesion[year] for year in years]
    plt.plot(years, values, 'o-', color='black', linewidth=2.5, 
            label='Average Across All Areas')
    
    # Plot top 5 policy areas with highest variance
    colors = plt.cm.tab10(np.linspace(0, 1, 5))
    for i, (policy, _) in enumerate(top_varying):
        policy_years = sorted(cohesion_data[policy].keys())
        policy_values = [cohesion_data[policy][year] for year in policy_years]
        plt.plot(policy_years, policy_values, 'o-', color=colors[i], 
                linewidth=1.5, label=policy)
    
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Ideological Cohesion (Lower = Higher In-Group Cohesion)', fontsize=12)
    plt.title('Ideological Group Cohesion Over Time by Policy Area', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Policy Area', fontsize=10)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('story_plots/7_ideological_cohesion.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a second visualization: Scatter plot of modularity vs average distance
    # This shows the relationship between overall polarization and community structure
    
    scatter_data = []
    for policy in polarization_metrics:
        for year in polarization_metrics[policy]:
            metrics = polarization_metrics[policy][year]
            if 'modularity' in metrics and 'avg_distance' in metrics:
                scatter_data.append({
                    'policy': policy,
                    'year': year,
                    'modularity': metrics['modularity'],
                    'avg_distance': metrics['avg_distance']
                })
    
    if scatter_data:
        scatter_df = pd.DataFrame(scatter_data)
        
        plt.figure(figsize=(10, 8))
        
        # Create categories based on policy area type (simplified example - adapt to your data)
        policy_categories = {}
        for policy in polarization_metrics:
            if 'budget' in policy.lower() or 'economic' in policy.lower() or 'tax' in policy.lower() or 'finance' in policy.lower():
                policy_categories[policy] = 'Economic'
            elif 'social' in policy.lower() or 'health' in policy.lower() or 'education' in policy.lower():
                policy_categories[policy] = 'Social'
            elif 'environment' in policy.lower() or 'climate' in policy.lower() or 'energy' in policy.lower():
                policy_categories[policy] = 'Environment'
            elif 'security' in policy.lower() or 'defense' in policy.lower() or 'military' in policy.lower():
                policy_categories[policy] = 'Security & Defense'
            elif 'migra' in policy.lower() or 'asylum' in policy.lower() or 'immigra' in policy.lower():
                policy_categories[policy] = 'Migration'
            elif 'agriculture' in policy.lower() or 'fish' in policy.lower() or 'food' in policy.lower():
                policy_categories[policy] = 'Agriculture'
            elif 'institu' in policy.lower() or 'constitu' in policy.lower() or 'governance' in policy.lower():
                policy_categories[policy] = 'Institutional'
            else:
                policy_categories[policy] = 'Other'
                
        # Add category to DataFrame
        scatter_df['category'] = scatter_df['policy'].map(policy_categories)
        
        # Define colors for categories
        category_colors = {
            'Economic': '#1f77b4',  # Blue
            'Social': '#ff7f0e',    # Orange
            'Environment': '#2ca02c',  # Green
            'Security & Defense': '#d62728',  # Red
            'Migration': '#9467bd',  # Purple
            'Agriculture': '#8c564b',  # Brown
            'Institutional': '#e377c2',  # Pink
            'Other': '#7f7f7f'  # Gray
        }
        
        # Plot each category with a different color
        for category, color in category_colors.items():
            subset = scatter_df[scatter_df['category'] == category]
            if not subset.empty:
                plt.scatter(subset['avg_distance'], subset['modularity'], 
                           color=color, alpha=0.7, s=60, label=category)
        
        # Add regression line
        from scipy.stats import linregress
        slope, intercept, r_value, p_value, std_err = linregress(
            scatter_df['avg_distance'], scatter_df['modularity']
        )
        x_line = np.linspace(scatter_df['avg_distance'].min(), scatter_df['avg_distance'].max(), 100)
        y_line = slope * x_line + intercept
        plt.plot(x_line, y_line, 'k--', alpha=0.5)
        
        # Add correlation coefficient to the plot
        plt.text(0.05, 0.95, f'Correlation: {r_value:.2f}', transform=plt.gca().transAxes, 
                fontsize=12, verticalalignment='top')
        
        plt.xlabel('Polarization (Average Distance)', fontsize=12)
        plt.ylabel('Modularity (Higher = More Distinct Communities)', fontsize=12)
        plt.title('Relationship Between Polarization and Community Structure', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend(title='Policy Category', fontsize=10)
        
        # Save the figure
        plt.tight_layout()
        plt.savefig('story_plots/8_polarization_structure.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    return cohesion_data, scatter_df if scatter_data else None

# Main function to generate all visualizations
def generate_all_visualizations(similarity_matrices):
    """Generate all visualizations for the EPG polarization story."""
    
    # Calculate polarization metrics
    polarization_metrics = calculate_polarization_metrics(similarity_matrices)
    
    # Part 1: Overview Heatmap with Enhanced Colors
    print("Creating overview heatmap with enhanced colors...")
    aggregate_matrix = create_overview_heatmap(similarity_matrices)
    
    # Part 2: Enhanced PCA Visualization with Correct Political Spectrum
    print("Creating enhanced PCA visualization...")
    pca_df = create_pca_visualization(aggregate_matrix)
    
    # Part 3: Interactive Time Series with Multiple Selection
    print("Creating interactive polarization time series...")
    filtered_policy_areas, avg_polarization_by_year = create_interactive_polarization_time_series(
        similarity_matrices, polarization_metrics
    )
    
    # Part 4: Policy Area Ranking (Keep Same)
    print("Creating policy area ranking...")
    sorted_areas, polarization_matrix = create_policy_area_ranking(
        polarization_metrics, avg_polarization_by_year
    )
    
    # Part 5: Polarization Trends Visualization (Fixed)
    print("Creating polarization trends visualization...")
    policy_trends = create_polarization_patterns_visualization(polarization_metrics)
    
    # Part 6: New Interesting Metrics Visualization
    print("Creating new interesting metrics visualization...")
    additional_metrics = create_interesting_metrics_visualization(polarization_metrics)
    
    # Create a comprehensive story document
    story_text = """
# European Parliament Voting Patterns: A Story of Polarization

This visual narrative explores how voting patterns in the European Parliament have evolved over time, with a focus on political polarization across different policy areas.

## Part 1: The Overview

The heatmap provides a broad overview of voting similarities between different European Parliament Groups (EPGs) across all years and policy areas. Bright red areas indicate high similarity (groups voting together), while blue areas show dissimilarity (groups voting against each other).

![Overview Heatmap](story_plots/1_overview_heatmap.png)

This visualization reveals the fundamental structure of EPG voting patterns. We can observe clear clustering of ideologically similar groups, with a general left-right spectrum visible. The progressive-left groups (GUE/NGL, Greens) show high internal similarity, as do the center-right and conservative groups (EPP, ECR). This confirms the expected ideological divisions in the Parliament.

## Part 2: EPG Positioning in Two Dimensions

Using Principal Component Analysis (PCA), we can visualize the relative positioning of EPGs based on their voting similarities:

![PCA Visualization](story_plots/2_pca_visualization.png)

The PCA plot reveals how EPGs are positioned relative to each other based on their voting behavior. The first principal component (x-axis) largely corresponds to the traditional left-right political spectrum, while the second component (y-axis) may represent other dimensions of political division such as attitudes toward European integration or social issues.

We can observe distinct clusters forming around traditional political families: the progressive left, social democrats, liberals, conservatives, and right-wing groups. The distance between groups in this visualization represents their voting dissimilarity.

## Part 3: Polarization Trends Over Time

How has polarization in the European Parliament evolved over time? The following time series shows the average distance between EPGs (our polarization metric) for selected policy areas:

![Polarization Time Series](story_plots/3_polarization_time_series.png)

The black line represents the average polarization across all policy areas, providing a reference point. We can observe that certain policy areas show significantly different polarization trends than others. Some areas have become increasingly polarized over time, while others have shown convergence.

For an interactive version of this visualization where you can select specific policy areas to compare, open the HTML file in the story_plots folder.

## Part 4: Which Policy Areas Are Most Polarizing?

Not all policy areas generate the same level of political division. The following visualizations rank policy areas by their average polarization:

![Policy Area Ranking](story_plots/4_policy_area_ranking.png)

The most polarized policy areas tend to relate to [specific types of policies: e.g., migration, economic policy, social issues], which aligns with our understanding of the most contentious issues in European politics.

In contrast, the least polarized areas are generally [specific types of policies: e.g., technical standards, research funding, infrastructure], where there tends to be more technocratic consensus.

## Part 5: Polarization Trends Across Policy Areas

Some policy areas are becoming more polarized over time, while others are showing increasing consensus. The following visualizations show the most dramatic trends in both directions:

![Polarization Trends](story_plots/5_polarization_trends.png)

The top panel shows policy areas that have become increasingly polarized over time, while the bottom panel shows areas where polarization has decreased. This helps us identify which issues are becoming more or less contentious in European politics.

We can also look at the overall distribution of trends across all policy areas:

![Trend Distribution](story_plots/6_trend_distribution.png)

This histogram shows how many policy areas are becoming more polarized versus less polarized. The vertical red line indicates zero change, while the green line shows the mean trend. This gives us a sense of the overall direction of polarization in the European Parliament.

## Part 6: Deeper Insights into Political Dynamics

Beyond simple polarization, we can explore more complex patterns in parliamentary voting behavior:

![Ideological Cohesion](story_plots/7_ideological_cohesion.png)

This visualization shows how cohesive ideological groups are in different policy areas over time. Lower values indicate that EPGs within the same ideological family (e.g., left, center-left) vote more similarly to each other than to EPGs from different families.

We can also examine the relationship between overall polarization and the formation of distinct voting communities:

![Polarization Structure](story_plots/8_polarization_structure.png)

This scatter plot reveals how polarization relates to the formation of distinct voting blocs. Each point represents a policy area in a specific year, colored by policy category. The correlation between these metrics tells us whether increased polarization tends to create more distinct voting communities or more fragmented voting patterns.

## Conclusion

Our visual analysis of European Parliament voting patterns reveals a complex landscape of political alignment and polarization. While the traditional left-right spectrum remains evident in voting behavior, polarization varies significantly across policy areas and time periods.

The findings suggest that [summarize key insights about polarization trends, their potential causes, and implications for European politics].

This analysis provides a foundation for understanding political dynamics in the European Parliament and how they have evolved over time. Future research could explore the causes of polarization in specific policy areas and the impact of external events on parliamentary voting patterns.
"""
    
    # Save the story text
    with open('story_plots/polarization_story.md', 'w') as f:
        f.write(story_text)
    
    print("All visualizations and story document created in the 'story_plots' folder.")

# You would call generate_all_visualizations(similarity_matrices) to create all plots and story

In [46]:
generate_all_visualizations(similarity_matrices)

Creating overview heatmap with enhanced colors...
Creating enhanced PCA visualization...
Creating interactive polarization time series...


Creating policy area ranking...
Creating polarization trends visualization...
Creating new interesting metrics visualization...
All visualizations and story document created in the 'story_plots' folder.
